# 附录 B： 变分法推断

【原文】Zhang Cheng， Butepage Judith，Kjellstrom Hedvig，Mandt Stephan； Advances in Variational Inference，2018

<style>p{text-indent:2em;2}</style>

## 1 问题提出

概率模型中的推断通常难以处理，通过对随机变量进行采样（ 即蒙特卡洛 ）的方法，能够为推断问题（例如，边缘似然的推断）提供近似解。大多数基于采样的推断算法属于`马尔可夫链蒙特卡罗 （ `MCMC` ) `方法。

基于采样的方法有几个重要的缺陷：

- 时间要求：尽管随机方法可以保证找到全局最优解，但前提是时间充足，而这在实践中通常是受限的，尤其是在大数据流行的当下。
- 收敛判断：目前尚没有好的方法能够判断采样结果与真实解到底有多接近，收敛状况仍然需要人工判断。
- 采样途径：为了找到一个效率足够高的解决方案，`MCMC` 方法需要选择合适的采样技术（例如，Metropolis-Hastings 、 HMC 、 NUTS 等），这种选择本身就是一门艺术。

为此，人们一直在寻找在保证适当精度条件下，比 `MCMC` 效率更高的后验推断方法，这就是 `变分推断（ Variational Inference ）` 方法。

## 2 变分推断

### 2.1 核心思想

根据贝叶斯概率框架，我们希望根据已有数据，来推断`模型参数`或`隐变量`的概率分布 $p$ ，进而能够（通过边缘化）完成后续预测任务；但当 $p$ 不容易表达、无法得到封闭形式解时，可以考虑寻找一个容易表达和求解的分布 $q$ 来近似 $p$ ，当 $q$ 和 $p$ 之间差距足够小时， $q$ 就可以代替真实分布 $p$ 并被用于执行后续任务。 当 $q$ 由若干参数 $\nu$ 定义或索引时，推断问题就进一步变成了寻找最佳 $\nu$ 的优化问题，优化目标是最小化 $q$ 和 $p$ 两个概率分布之间的差距。其中， $q$ 被成为 **变分分布**， $\nu$ 被称为 **变分参数**。

> 变分推断的精髓是将 “求真实分布” 的推断问题，转变成了 “求近似分布” 的问题，并进一步转变成了 “缩小差距” 的优化问题。由于 $q$ 完全由人工定义，因此具备很大的灵活性。

以下图为例进行说明，黄色分布为目标分布 $p$ ，很难求解，但它看上去有些像高斯，于是可以尝试从高斯分布族 $\mathcal{Q}$ 中（此处以一个红色高斯和一个绿色高斯为例，其变分参数为均值 $\mu$ 和 方差 $\sigma^2$ ），寻找和 $p$ 最像的（可以用“重叠面积比率最大” 或 “差异最小” 来度量 ） 那个 $q$ ， 作为 $p$ 的近似分布。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108085400-e452.webp)

> 图 1：单变量的真实分布与近似分布。 以单变量举例只是为了可视化理解，真实情况通常是具有很多个维随机变量的联合分布（如：深度神经网络可能包含几百万个参数维度），而且涉及多维空间中的多峰分布。

变分方法与采样方法的主要区别是：

- 变分方法几乎永远找不到全局最优解

- 可以知道自身收敛情况，甚至可以对其准确性有限制

- 更适合于随机梯度优化、多处理器并行处理以及使用 GPU 加速

虽然采样方法的历史非常悠久，但随着大数据发展，变分方法一直在稳步普及，并成为目前使用更为广泛的推断技术。
 
 ### 2.2 进一步加深理解

可以从以下五点加深对变分推断的理解：

- **模型**：变分推断需要设计一个模型 $p(\mathbf{z, x})$ ，其中 $\mathbf{x}$ 为观测数据， $\mathbf{z}$ 为模型参数或/和隐变量（ 模型参数通常是机器学习的提法，后面将统一采用概率模型中的隐变量来表示 ）。模型来自专家个人的业务知识和归纳偏好，通常可以用概率图模型表达（ 注：一张概率图表达的正是图中所有随机变量节点的联合分布 ），而数据则来自实际观测。
- **目的**：基于观测数据 $\mathbf{x}$ 确定模型中隐变量 $\mathbf{z}$ 的值，以便进一步完成后续预测或决策任务。
- **问题**：各种不确定性因素导致上述隐变量无法得到确切的结果，通常只能给出其概率分布 $p(\mathbf{z \mid x})$ ，而最大的问题在于这个分布可能很复杂，无法直接给出封闭形式的解。
- **原理**：变分推断将人为构造一个由参数 $\boldsymbol{\nu}$ 索引（或定义）的、与$\mathbf{x}$ 无关的概率分布 $q(\mathbf{z};\boldsymbol{\nu})$ ，并通过一些方法或技巧来优化 $\boldsymbol{\nu}$，使 $q(\mathbf{z};\boldsymbol{\nu})$ 能够近似并且代替复杂的真实分布 $p(\mathbf{z \mid x})$ 。
- **途径**：通过一些优化算法，渐进地调整 $\boldsymbol{\nu}$ 以缩小 $q(\mathbf{z};\boldsymbol{\nu})$ 和 $p(\mathbf{z \mid x})$ 之间的差异直至收敛。

#### 2.2.1 变分推断基于贝叶斯模型

变分推断是基于模型实施的，也就是说必须 “先有模型后又推断” 。简单说，专家利用其领域知识和归纳偏好，给出一个模型假设 $p(\mathbf{z, x})$ ，其中包括隐变量 $\mathbf{z}$ 和观测变量 $\mathbf{x}$ ，以及变量相互之间可能存在的依赖关系；然后基于该模型和观测数据推断其中隐变量的分布 $p(\mathbf{z} \mid \mathbf{x})$。

> 注： 此处 `隐变量` 和 `观测变量` 均采用黑体符号，表示其为向量形式，即代表了不止一个随机变量。

为了理解隐变量和观测变量之间的关系，一种比较易于理解的表达方式是建立 “观测变量的生成过程” ，即认为`观测变量`是从某个已知的、由`隐变量`组成的结构中生成出来的。

以高斯混合模型为例（见下图）。图中为我们观测到一个数据集，它实际上是从由 5 个相互独立的高斯组合而成的一个混合分布中采样的结果。如果从单个数据点出发，考虑其生成过程，可以分为两步：

- 首先从 5 个类别组成的离散型分布中抽取一个类别样本（比如粉红色类别）

- 然后从类别对应的高斯分布中抽取一个样本生成相应数据点。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108085412-fdef.webp)

> 图 2 ： 5 个组份构成的高斯混合模型示意图

在上例中，可以发现隐变量有：

- 5 个高斯分布的均值 $\boldsymbol{\mu}$ 和方差 $\boldsymbol{\sigma^2}$ （ 均为长度为 5 的向量 ）
- 每个数据点 $i$ 所属类别 $\mathbf{c}_i$ （独热向量形式）

上述变量 $\boldsymbol{\mu}$、$\boldsymbol{\sigma}^2$ 和 $\mathbf{c}$ 一起构成了隐变量 $\mathbf{z}$ ，而且三者之间可能存在一定的依赖关系。

#### 2.2.2 问题本质是基于观测数据推断隐变量的分布

后验概率 $p(\mathbf{z|x})$ 的物理含义是：基于现有观测数据 $\mathbf{x}$ ，推断隐变量 $\mathbf{z}$ 的概率分布。 

对于上面的高斯混合模型来说，变分推断的目的就是求得隐变量 $\mathbf{z} = \{\boldsymbol{\mu},\boldsymbol{\sigma^2}, \mathbf{c} \}$ 的后验分布 $p(\mathbf{z} \mid \mathbf{x})$ 。根据贝叶斯公式，$p(\mathbf{z} \mid \mathbf{x}) = p(\mathbf{z,x}) / p(\mathbf{x})$ 。 依据专家设计的生成过程，能够写出分子中联合分布 $p(\mathbf{z,x})$ 的表达式，但分母中的边缘似然 $p(\mathbf{x})$ 是一个很难处理的项。当 $\mathbf{z}$ 为连续型时，边缘似然 $p(\mathbf{x})$ 需要在隐变量 $\mathbf{z}$ 的概率空间内求 $p(\mathbf{z,x})$ 的积分（ 即做边缘化处理 ）；当 $\mathbf{z}$ 为离散型时，需要对所有可能的 $\mathbf{z}$ 求和；而积分（和求和）的计算复杂性随着样本数量的增加会呈指数增长。

> 注解： 细心的读者会发现，求 `边缘似然` 和求 `预测分布` 都是在做边缘化，区别仅在于 `边缘似然` 是基于先验空间做边缘化，而 `预测分布` 是在后验空间做边缘化。

#### 2.2.3 变分推断的关键是构造 `变分分布` 

变分推断需要构造`变分分布` $q(\mathbf{z}; \boldsymbol{\nu})$，并通过优化调整 $\boldsymbol{\nu}$，使 $q(\mathbf{z}; \boldsymbol{\nu})$ 更接近真实后验 $p(\mathbf{z \mid x})$ 。

在`变分分布` $q(\mathbf{z}; \boldsymbol{\nu})$ 中， $\mathbf{z}$ 为隐变量，$ \boldsymbol{\nu}$ 是控制 $q$ 形态的参数（例如：假设 $q$ 为高斯分布，则 $\boldsymbol{\nu}$ 为均值和方差 ， 而对于泊松分布，则 $\boldsymbol{\nu}$ 为二值概率）。因此构造`变分分布` $q$ 分为两步：

- 首先是概率分布类型的选择。通常依据 $p$ 的形态由专家给出，例如在上述高斯混合模型中，假设目标分布 $p$ 服从多元高斯分布，则构造 $q$ 时通常依然会考虑高斯分布。

- 其次是概率分布参数的确定。该确定过程是一个逐步优化迭代的过程，通常经过渐进调整 $\boldsymbol{\nu}$ 的值 ，使 $q$ 逐渐逼近 $p$ 。

#### 2.2.4 变分推断是一个优化问题

变分推断是一个优化问题，其最直观的优化目标是最小化 `KL 散度` ，但实际可操作的优化目标是最大化证据下界 `ELBO` 。

直观地理解，变分推断的优化目标很明确，就是最小化 $q$ 和 $p$ 之间的差距。信息论为我们提供了一个度量两个分布之间差距的数学工具： `KL 散度` 。但不幸的是， `KL 散度` 的计算并不简单，因为其计算表达式中仍然包含难以处理的边缘似然项（即贝叶斯公式中的分母积分项，也称证据，在一些数学形式中也被称为边缘似然）。为此，有人提出了可操作的优化目标 --- 证据下界 `ELBO` （见 2.4 节）。

在证据下界 `ELBO` 的计算表达式中，只包括联合分布 $p(\mathbf{z, x})$ （ 即贝叶斯公式的分子项 ） 和 `变分分布` $q(\mathbf{z}; \boldsymbol{\nu})$ ，从而摆脱了难以处理的边缘似然项。数学推导表明，在给定观测数据后，最大化 `ELBO` 等价于最小化 `KL 散度` 。 也就是说，`ELBO` 的最大化过程结束时，获得的输出 $q(\mathbf{z}; \boldsymbol{\nu^\star})$ 就是我们寻求的 `最佳变分分布`。

#### 2.2.5 变分推断的形象化解释

下图为变分推断的形式化示意图。图中大圈表示了一个分布族，由参数 $\boldsymbol{\nu}$ 索引。我们也可以理解为，大圈是分布 $q$ 的参数 $\boldsymbol{\nu}$ 的取值空间。大圈中每个点均表示不同 $\boldsymbol{\nu}$ 值对应的分布 $q$ 。例如，在高斯混合模型的例子中， $q$ 属于高斯分布族，则不同的均值和方差，代表了不同的分布 $q$ 。从 $\boldsymbol{\nu}^{init}$ 到 $\boldsymbol{\nu}^*$ 的路径表示变分推断的优化迭代过程，通过优化更新 $\boldsymbol{\nu}$ 值来缩小 $q$ 与 $p$ 之间的差距，而这个差距通常采用 `KL 散度` 来衡量。路径终点 $\boldsymbol{\nu}^*$ 对应的分布 $p$ 与真实后验 $p(z \mid x)$ 之间应当是某种距离测度上的最小值。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108085528-fa88.webp)

> 图 3 ：变分推断示意图。图中隐变量为 $\mathbf{z}$，包含局部变量和全局变量。$\nu$ 对应了所有变分参数。

### 2.3 概率图表示

#### 2.3.1 贝叶斯概率框架

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108085443-66c1.webp)

> 图 4 ：贝叶斯概率框架示意图

上图是贝叶斯统计问题的一般求解流程图。领域专家拥有知识可以用来建模，并且存在需要解答的问题。他们依据拥有的知识，给出带有归纳偏好的合理假设，构建出数据的生成过程模型 (Generative Processing Model)。模型中主要包括隐变量、观测变量以及它们之间的依赖关系。利用该模型，我们希望通过对观测数据的处理，从中挖掘出有价值的模式，然后实现各式各样的应用。

还是以高斯混合模型为例，可以将其生成过程形式化为：

$$
\begin{aligned}
 \boldsymbol{\mu}_{k} & \sim \mathcal{N}\left(0, \sigma^{2}\right), & & k=1, \ldots, K, &（先验）\\
\mathbf{c}_{i} & \sim \text { Categorical }(1 / K, \ldots, 1 / K), & & i=1, \ldots, n, &（先验）\\
x_{i} \mid \mathbf{c}_{i}, \boldsymbol{\mu} & \sim \mathscr{N}\left(\mathbf{c}_{i}^{\top} \boldsymbol{\mu}, 1\right) & & i=1, \ldots, n &（似然）
\end{aligned}
$$

该模型混合了 $K$ 个相互独立的高斯组份（ 注意， $K$ 是超参数而非隐变量 ），模型中的隐变量包括数据点所在的类别 $ \mathbf{c}_i$，各类别组份的均值 $\boldsymbol{\mu}_k$ 和方差 $ \boldsymbol{\sigma}_k^2$ （为简化问题，此处假设 $ \boldsymbol{\sigma}_k^2$ 为常数 1 ）。

设所有高斯组份的均值参数 $\boldsymbol{\mu}_k$ 的先验为 0 均值的同方差（$\sigma^2$）高斯，如第一行所示；类别参数 $ \mathbf{c}_i$ 的先验为均匀的类别分布（因为缺乏关于类别的先验知识），如第二行所示。假设所有样本数据都是从某个方差为 $1$ 的高斯混合模型中随机生成的，则任一数据点 $x_i$ 的生成过程可以通过似然表达出来（如第三行所示），分为两步 ：

- 第一步，从类别分布中随机抽取一个离散型的类标签 $ \mathbf{c}_i$ （此处采用独热变量形式）；

- 第二步，从类标签 $ \mathbf{c}_i$ 对应的均值为 $\mu = \mathbf{c}_{i}^{\top} \boldsymbol{\mu}$ 、方差为 $1$ 的高斯分布中，随机抽取一个数据点 $x_i$。

更细致的举例说明见图 2，五个分布用不同颜色表示，代表五个类别。每次从中选择一个类，如第三类粉红色，$\mathbf{c}_i=\{0，0，1，0，0\}$ ，然后从第三类对应的高斯分布中抽取 $x_i$ ，其均值为 $\mathbf{c}_i \cdot \boldsymbol{\mu}=\mu_3$。该点大概率出现在粉红色类覆盖的区域内。

#### 2.3.2 利用概率图表达随机变量的联合分布和依赖关系

概率图模型是表达随机变量之间结构和关系的有力工具（参见概率图理论文献）。上述高斯混合模型的形式化生成过程，在概率图模型中体现为三个随机变量及它们之间的依赖关系（见下图），其中高斯组份的均值 $\boldsymbol{\mu}$ 和数据点所属类别 $\mathbf{z}$ 为隐变量，用白色圈表示；$\mathbf{x}_i$ 为观测变量，用灰色圈表示。此外，可以将隐变量的集合进一步划分为全局隐变量和局部隐变量，例如，图中 $\boldsymbol{\mu}$ 为全局隐变量，其作用发挥在所有数据上，而 $\mathbf{z}_i$ 为局部隐变量，只和数据点 $x_i$ 相关，与其他点无关。通常将局部变量用矩形框包围起来，以便与全局变量分隔开来。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108085458-b4a9.webp)

> 图 5：高斯混合模型的概率图（1）白圈为隐变量，灰圈为观测变量；（2）矩形框内为局部变量，外部为全局变量；（3）矩形框表示其内部的变量 $z_i$ 和 $x_i$ 独立重复了 n 次；（4）箭头表示随机变量之间的条件依赖关系。

基于上述概率图，可以解析地写出所有随机变量的联合概率分布（详情参见概率图原理）：

$$
p(\boldsymbol{\mu}, \mathbf{z}, \mathbf{x})=p(\boldsymbol{\mu}) \prod_{i=1}^{n} p\left(z_{i}\right) p\left(x_{i} \mid z_{i}, \boldsymbol{\mu}\right)
$$

那么在上述高斯混合模型中，到底需要变分方法推断什么？

根据统计机器学习框架，此例中观测数据 $\mathbf{x}$ 由无标签的数据点构成，我们需要根据这些数据点来推断 $\mathbf{z}$ 和 $\boldsymbol{\mu}$ 的分布。即给定观测数据 $\mathbf{x}$ 时，推断隐变量的后验概率。根据基础概率公式，其形式化描述为：

$$
p(\boldsymbol{\mu}, \mathbf{c} \mid \mathbf{x})=\frac{p(\boldsymbol{\mu},\mathbf{c} , \mathbf{x})}{p(\mathbf{x})}
$$

式中，分子项为 $p(\boldsymbol{\mu},\mathbf{c} , \mathbf{x})$ ， 可根据概率图模型理论写出表达式，但分母 $p(\mathbf{x})$ 涉及所有数据点的积分问题，是难以处理的。因此，后验分布 $p(\boldsymbol{\mu}, \mathbf{c} \mid \mathbf{x})$ 很难直接求得，需要用近似的、可处理的近似分布 $q(\boldsymbol{\mu},\mathbf{c})$ 来代替。

#### 2.3.3 推广到更一般的形式

把上述问题梳理一下，推广到更一般的形式，可以用来描述各种各样的贝叶斯统计模型：

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108085516-aec3.webp)

> 图 6：包含全局隐变量和局部隐变量的一般性概率图表示

在贝叶斯公式中，后验概率的分子项为联合概率分布，可通过概率图得出表达式（图 6 下）；但分母中的边缘似然项依然难以处理（ 见下式 ）。即便隐变量是离散型变量，存在 $K$ 个可能值，其计算复杂度也为 $O(K^n)$ ，意味着其计算复杂度会随着数据量 $n$ 的增长呈指数增长。

$$
p(\beta, z \mid x)=\frac{p(\beta, z, x)}{p(x)}=\frac{p(\beta, z, x)}{\iint_{\beta z} p(x) d \beta d z}
$$

### 2.4 目标函数的选择

要将推断转化为优化问题，需要选择或者构造一个`变分分布族` $\mathcal{Q}$ ，并设定合适的优化目标 $J(q)$ 。 关于分布族的讨论放在下一节，此处先讨论优化目标 $J(q)$ 。该目标需要捕获 $q$ 和 $p$ 之间的差距（或反之，相似性），而信息论为我们提供了一个直观并且好理解的工具，被称为 **KL（ Kullback-Leibler ）散度**。

从形式上理解， `KL 散度` 指两个分布之间的差异。 $q$ 和 $p$ 之间离散形式的 `KL 散度` 被定义为：

$$
KL(q \| p) = \sum_x q(x) \log \frac{q(x)}{p(x)}
$$

在信息论中，此函数用于测量两个分布中所包含信息的差异大小。 `KL 散度` 具有以下性质，使其在近似推断任务中特别有用：

- 对所有 $q$ 和 $p$ ，都有 $KL(q\|p) \geq 0$ 。 
- 当且仅当 $q = p$ 时，$KL(q\|p) = 0$ 。 

请注意： `KL 散度` 不具备可逆性（或对称性），即 $KL(q\|p) \neq KL(p\|q)$ 。这也是称之为`散度`，而不是`距离`的原因，因为距离是双向对称的。

 `KL 散度` 虽然很好理解，但是是否能够直接使用 `KL 散度` 进行变分推断呢？ 我们有必要针对一般化形式的目标分布 $p$ 做一下分析。 

假设 $p$ 为一个泛化（离散、简单）的、具有如下形式的无方向概率模型（其实该形式是统计机器学习框架中后验分布的一般化形式，也是变分推断面临的主要场景）：

$$
p(x_1,\ldots,x_n; \theta) = \frac{\tilde p(x_1,\ldots,x_n ; \theta)}{Z(\theta)} =\frac{1}{Z(\theta)} \prod_{k} \phi_k(x_k; \theta)
$$

> 注：此处假设了真实分布 $p$ 的后验可以被因子分解的情况，是变分推断中比较常见的一种假设，详情见后面的平均场变分推断部分。

其中 $k$ 为模型中因子的数量（有关因子的概念，参见无向概率图或马尔科夫随机场相关资料）， $\phi_k(\cdot)$ 为因子， $Z(\theta)$ 为归一化常数或边缘似然。假设该形式能够涵盖所有可能的目标分布， 则在该一般化形式下，归一化常数 $Z(\theta)$ 面临着难以处理的问题。由于 `KL 散度` 的计算需要用到 $p(x)$ ，因此 $Z(\theta)$ 的存在导致无法直接使用 $KL(q\|p)$ 做为优化目标。

考虑到归一化常数虽然难以处理，但在观测数据确定的情况下是常数，因此，可以考虑仅处理形式上与 `KL 散度` 相似的未归一化分子项 $\tilde p(x) = \prod_{k} \phi_k(x_k; \theta)$ ，采用对数形式，即将优化目标调整为：

$$
J(q) = \sum_x q(x) \log \frac{q(x)}{\tilde p(x)}
$$

上述目标函数中， $q(x)$ 来自人工构造、 $\tilde p (x)$ 来自于（ 数据概率生成过程中的 ）似然和先验，各项均可处理。 更重要的是，其还具有以下重要性质：

$$
\begin{align*} 
J(q) &= \sum_x q(x) \log \frac{q(x)} {\tilde p(x)} \\ 
&= \sum_x q(x) \log \frac{q(x)}{p(x)} - \log Z(\theta) \\ 
&= KL(q\|p) - \log Z(\theta) 
\end{align*}
$$

也就是说，新的目标函数 $J(q)$ 是 `KL 散度` 与`对数边缘似然`之差，而根据 `KL 散度` 的性质， $KL(q\|p) \geq 0$ ，则重排公式可得：

$$
\log Z(\theta) = K算法 1L(q\|p) - J(q) \geq -J(q)
$$

该式表明，新目标函数的相对值 $-J(q)$ 是`对数边缘似然` $\log Z(\theta)$ 的下界。在大多数应用场景中， 边缘似然 $Z(\theta)$ 都有着特定的解释。例如贝叶斯框架下，在给定观测数据 $D$ 的情况下，我们可能想计算随机变量 $x$ 的边缘概率 $p(x \mid D) = p(x,D) / p(D)$ 。在这种情况下，最小化 $J(q)$ 等价于最大化`对数边缘似然` $\log p(D)$ 的下界。正是因为如此， $-J(q)$ 被称为变分下界或证据下界（ `ELBO` ），这种关系经常以如下不等式形式表示：

$$
\log Z(\theta) \geq \mathbb{E}_{q(x)} [ \log \tilde p(x) - \log q(x) ]=\ ELBO
$$

> 注：依据公式， $J(q)$ 可以视为在分布 $q(x)$ 上求 $\log \frac{q(x)} {\tilde p(x)}$ 的期望值。

最关键的是，对数边缘似然 $\log Z(\theta)$ 和变分下界 $-J(q)$ 的差，正好是 `KL 散度` 。 因此，最大化变分下界等效于通过挤压 $-J(q)$ 和 $\log Z(\theta)$ 之间的差，实现最小化 $KL(q\|p)$ 。 

--- 

**讨论：**

刚刚为变分推断重新定义了优化目标，并且表明，最大化下界等效于最小化散度 $KL(q\|p)$ 。 

回想之前说过的 $KL(q\|p) \neq KL(p\|q)$ ; 当 $q = p$ 时，两种散度都等于零，但当 $q \neq p$ 时，两者的值不同。 这就提出了一个问题：为什么选择其中一个而非另一个，它们有何不同？

也许最重要的区别是计算效率：优化 $KL(q\|p)$ 时，涉及关于 $q$ 求期望；而优化 $KL(p\|q)$ 则需要关于 $p$ 求期望，而这又是难以处理甚至无法评估的。

但当近似分布族 $\mathcal{Q}$ 不包含真实分布 $p$ 时，选择 $KL(p\|q)$ 会影响返回的解。 通过观察会发现， 如果 $p(x) = 0$ 并且 $q(x) > 0$ 时， $KL(q\|p)$ 是无限的 （ 该散度有时被称为 I 投影或信息投影 ） :

$$
KL(q\|p) = \sum_x q(x) \log \frac{q(x)}{p(x)}
$$

这意味着，如果 $p(x) = 0$ ，则必须有 $q(x) = 0$ 。 我们称： $KL(q\|p)$ 对于 $q$ 是零强制的，并且它通常会低估 $p$ 的支持。 

另一方面，当 $q(x) = 0$ 并且 $p(x) > 0$ 时， $KL(p\|q)$ 是无限的 （ 该散度被称为 M 投影或矩投影）。 因此，如果 $p(x) > 0$ ，则必须有 $q(x) > 0$ 。 此时，我们称： $KL(p\|q)$ 对于 $q$ 是零避免的，并且它通常会高估 $p$ 的支持。 

下图以图形方式说明了该现象。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108081010-6d1d.webp)

> 图 7：将单峰近似分布 $q$（红色）拟合到多峰 $p$（蓝色）。 (a) 使用 $KL(p||q)$ 会导致 $q$ 试图覆盖两个峰。 (b）(c) 使用 $KL(q||p)$ 会迫使 $q$ 选择 $p$ 的其中一个峰。

鉴于两种散度的上述性质，我们经常称 $KL(p\|q)$ 为包容性 `KL 散度` ， 而 $KL(q\|p)$ 是独占性的 `KL 散度` 。

---

小结：

- 由于边缘似然 $Z$ 的存在，使得直接计算 `KL 散度` 难以处理
- 采用等价的最大化 `ELBO` 来代替最小化 `KL 散度` 作为新的优化目标
- `ELBO` 中只包含`变分分布` $q$ 和目标分布的分子项 $\tilde p$ ，并且两者都可计算，进而可以作为真正的优化目标
- $KL(q\|p)$ 和 $KL(p\|q)$ 虽然都是散度，也都可以作为优化目标，但作为目标时两者达到的效果截然不同。对于多峰后验，前者倾向于选择能够覆盖其中某个峰的`变分分布`，而后者倾向于选择能够覆盖多个峰的`变分分布`。

## 3 经典方法 --- 平均场、指数族与坐标上升算法

### 3.1 平均场近似的数学推导

上一节曾经剃刀，在构造`变分分布`时，需要在 $q$ 的表达能力和简单易处理性之间做权衡。其中一种常见的选择是使用**完全因子分解**的方法来构造分布，也被称为平均场分布。

平均场近似假设所有隐变量相互独立，进而简化了推导。当然，这种比较强的独立性假设会导致不太准确的近似，特别是当后验中的随机变量存在高度依赖时尤其如此。因此，有人研究更为准确的近似方法，本文第 6 节会讨论此类更具表现力的`变分分布`。

`平均场变分推断 ( MFVI )` 起源于统计物理学的 [平均场理论](https://ieeexplore.ieee.org/book/6267422)。在平均场近似中，基于独立性假设，`变分分布`被分解为各因子分布的乘积，而每个因子由其自身的变分参数控制：

$$
q(\mathbf{z};\boldsymbol{\lambda}) = \prod_{i=1}^N q(z_i ; \lambda_i)
$$

为了符号简单，我们将在本节的其余部分省略了变分参数 $λ$ 。现在看一下如何在平均场假设下最大化 `ELBO` 。

完全因子分解的`变分分布`可以通过简单的迭代更新来做优化。为了看到这一点，我们可以将注意力放在更新与隐变量 $z_j$ 相关的变分参数 $λ_j$ 上。首先，将平均场分布引入 `ELBO` 的计算公式后，可以得到如下表示：

$$
\begin{align*}
\mathscr{L}= \int q\left(z_{j}\right) \mathbb{E}_{q\left(z_{\neg j}\right)}\left[\log p\left(z_{j}, \boldsymbol{x} \mid \boldsymbol{z}_{\neg j}\right)\right] d z_{j} \\ -\int q\left(z_{j}\right) \log q\left(z_{j}\right) d z_{j}+c_{j} \tag{6}
\end{align*}
$$

> 注： 此处采用了连续隐变量的表达方式，对于离散型隐变量，积分符号要改为求和符号。

$\boldsymbol{z}_{\neg j}$ 表示集合 $\boldsymbol{z}$ 中除 $z_{j}$ 以外的变量集合；常数 $c_{j}$ 包含所有相对于 $z_{j}$ 可视为常数的项（ 例如：与 $z_{\neg j}$ 相关的熵项，由于 $z_{\neg j}$ 相对于 $z_{j}$ 是常数，因此该熵项可视为常数项 ）。至此，我们将完整的期望分解为了两个部分：一是关于 $\boldsymbol{z}_{\neg j}$ 的内部期望；另一个是关于 $z_{j}$ 的外部期望。

公式（ 6 ）采用负 $\mathrm{KL}$ 散度的形式，对于变量 $z_j$ 最大化

$$
\log q^{*}\left(z_{j}\right)=\mathbb{E}_{q\left(z_{\neg j}\right)}\left[\log p\left(z_{j} \mid \boldsymbol{z}_{\neg j}, \boldsymbol{x}\right)\right]+\text { const } \tag{7}
$$

对这个结果求幂并做归一化，产生：

$$
\begin{aligned}
q^{*}\left(z_{j}\right) & \propto \exp \left(\mathbb{E}_{q\left(z_{\neg j}\right)}\left[\log p\left(z_{j} \mid z_{\neg j}, \boldsymbol{x}\right)\right]\right) \\
& \propto \exp \left(\mathbb{E}_{q\left(z_{\neg j}\right)}[\log p(z, \boldsymbol{x})]\right) \tag{8}
\end{aligned}
$$

使用公式（ 8 ）所示，可以针对每个隐变量迭代地更新变分分布，直至其收敛。类似的更新也构成了变分消息传递算法 [216] 的基础。

有关平均场近似及其几何解释的更多详细信息，请读者参阅 [14] 和 [205]。

### 3.2 指数族分布的适用性

变分推断的另一个重要的技术点在于`变分分布族` $\mathcal{Q}$ 的选择。在高斯混合模型中，我们假设目标分布 $p$ 是高斯的，因此将`变分分布` $q$ 构造为高斯分布族，会更容易逼近 $p$ 。 

但对于更为复杂的目标分布怎么办？答案是：**使用更广泛的指数族分布来构造！！！**

当 $p$ 属于指数族分布时，数学上已经证明，都可以将其近似分布 $q$ 构造为相同类型的指数族分布。指数族分布具有一些非常好的性质，允许很巧妙地简化`自然梯度`的推导（ `自然梯度`的概念见 4.2 节）。最重要的是，指数族分布非常宽泛，基本涵盖了常见的分布形态，如：高斯分布、$\chi ^2$ 分布、伯努利分布、指数分布、贝塔分布、伽马分布、泊松分布等。

当 $p$ 确实不属于指数族时，平均场变分分布 $q$ 可能永远无法近似 $p$ ，这是变分推断的一个缺陷，即最终计算得出的结果是一个难以提前估计的近似。

### 3.3 平均场推断优化算法 -- 坐标上升法

**（1）概率图中的平均场变分推断**

在构造 $q$ 时，除了对分布族做选择之外，还需要考虑不同随机变量之间的依赖关系问题。其中一种最简单的近似，就是假设隐变量可以被分解为多个因子，而因子之间相互独立，此即平均场的原理。

根据概率图模型，此时隐变量的联合分布可被分解为多个因子分布的乘积（见下图）。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108085537-3530.webp)

> 图 8：`LDA` 模型及其完全独立因子分解的概率图表示

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108085543-3d28.webp)

> 图 9：完全因子分解后的平均场变分族

在上述因子分解形式的近似中，`完全因子分解`是使用最广泛的一种构造形式，该分解形式中，每个隐变量都有自己的独立分布，而与其他隐变量无关。数学形式为： 

$$
q(z) = q_1(z_1) q_2(z_2) \cdots q_n(z_n) 
$$

其中所有的 $q_i(z_i)$ 都是一元隐变量上的分布。显而易见地，当 $z_i$ 为离散型变量时， 则 $q_i(z_i)$ 为类别分布（ `Catalogical Distribution` ），通常描述为一维对照表形式。

事实表明，对于 $\mathcal{Q}$ 的这种构造很容易进行优化，并且工作地出奇得好。在优化变分下界时，这种因子分解的方法也许是目前最流行的一种选择。

**（2）平均场变分推断的优化算法**

依据变分推断的基本原理，平均场变分推断需要解决如下优化问题：

$$
\min_{q_1, \ldots, q_n} KL(q || p) \ 或 \ \max_{q_1, \ldots, q_n} ELBO(q || p)
$$

解决此优化问题的标准方法是在 $q_j$ 上对 `KL 散度` 实施坐标下降法做最小化，或对证据下界 `ELBO` 实施坐标上升法做最大化。

坐标下降法（或上升法）的原理也比较简单：每次仅更新一个维度的变分参数。即在 $j=1,2,\ldots,n$ 上做迭代，对于每一个 $j$ ，在保持其他维度随机变量 $q_{-j} = \prod_{i \neq j} q_i$ 固定（所谓固定指保持其他维度的变分参数不变，或视为常数 ），仅关于 $q_j$ 来优化 $KL(q\|p)$ 或 `ELBO`；所有维度 $j$ 遍历一遍后完成一次完整更新。

> 注解：  此处坐标下降法（或上升法）以完全因子分解为例。

最有趣的是：**单一坐标的优化问题具有简单的封闭形式解析解**：

$$
 \log q_j(z_j) \gets \mathbb{E}_{q_{-j}} \left[ \log \tilde p(z) \right] + \textrm{const}
$$

注意上述公式两侧都包含一元隐变量 $z_j$ 的函数，只是用右侧的另一个函数取代了 $q(z_j)$ 。右式中的常数项为新分布的归一化常数（依据前面基本原理的介绍，该常数项中应当包含其他与 $z_j$ 无关的隐变量相关的项 ），此外还包含一个因子总和的期望：

$$
\log \tilde p(z) = \sum_k \log \phi(z_k)
$$

根据无向概率图模型（即马尔科夫随机场理论），只有在变量 $z_j$ 的马尔科夫毯内的因子，才是 $z_j$ 的函数 ，马尔科夫毯之外的其余因子对于 $z_j$ 都可视为常数，可以纳入常数项 $\textrm{constant}$。

这样能够大量减少期望公式中的因子数量；当 $z_j$ 的马尔科夫毯很小时，可以比较容易给出 $q(z_j)$ 的解析公式。 例如，假设变量 $z_j$ 是具有 $K$ 个可能值的离散型变量，并且在 $z_j$ 的马尔科夫毯中有 $F$ 个因子和 $N$ 个变量，则计算期望值的时间复杂度为 $O(K F K^N)$：即对于每个 $z_j$ 的 $K$ 种可能的取值，需要对 $N$ 个变量的所有 $K^N$ 次赋值以及 $F$ 个因子求和。

坐标下降过程的结果是：

依据 $KL(q\|p)$ 迭代地拟合了用于近似 $p$ 的完全因子式 $q(z) = q_1(z_1) q_2(z_2) \cdots q_n(z_n) $ ； 而坐标下降的每一步，都增加了变分下界，使其向 $\log Z(\theta)$ 进一步收紧。 最终的因子 $q_j(z_j)$ 不会完全等于真正的边缘分布 $p(z_j)$ ，但对于许多实际目的，它们通常表现足够好，如确定 $z_j$ 的峰值： $\max_{z_j} p(z_j)$ 。 

**（3）以高斯混合模型为例**

还是用高斯混合模型的例子，根据概率图模型有：

$$
p(\boldsymbol{\mu},\mathbf{c},\mathbf{x})=p(\boldsymbol{\mu})\prod_{i=1}^{n} p(c_i)p(x_i \mid c_i,\boldsymbol{\mu})
$$

该模型中涉及的隐变量为 $\mathbf{z} = \{\boldsymbol{\mu},\mathbf{c} \}$ ，其中 $\boldsymbol{\mu} =\{ \mu_k\}, k=1...5$ 为 5 个高斯组份的均值；$\mathbf{c}$ 为所有数据点的类别向量。$\boldsymbol{\mu}$ 作为全局变量，其`变分分布`被构造为高斯分布 $q(u_k;m_k,S_k)$ ， $\nu_\mu = \{m_k,S_k\},k=1...5$ 为需要优化求解的变分参数。 $c_i$ 为局部变量，其`变分分布`的构造形式为多项分布 $q(c_i ;\psi_i)$ ，其中 $\nu_c=\{ \psi_i \}, i=1...n$ 为控制每个数据点类别的变分参数（注意 $\psi_i$ 为向量）。

按照平均场近似方法，各因变量之间相互独立，则`变分分布` $q(\boldsymbol{\mu},c)$ 可被构造为：

$$
p(\boldsymbol{\mu},\mathbf{c} \mid \mathbf{x}) \approx q(\boldsymbol{\mu},c)=\prod_{k=1}^{K} q(\mu_k;m_k,S_k^2) \prod_{i=1}^{n}q(c_i ;\psi_i)
$$

## 4 提升可扩展性 --- 随机变分推断

上节中讲解了`平均场变分分布`的构造原理，并介绍了采用坐标上升法对其做优化的算法。但该方法存在两个方面的问题：

- 最早提出的坐标上升法没有考虑大样本问题

- 梯度下降并非最优方向，收敛速度较慢

随机变分推断方法针对上述问题，分别进行了处理。下面分别介绍。

### 4.1 如何适应大数据 ？

首先看一下最原始的坐标上升方法，其中所有隐变量同等对待，按照维度顺序逐一更新一遍，效率较低。为此，有学者对其做了改进，将隐变量分为两种类型：仅关系单个数据点的**局部变量**，和关系到所有数据点的**全局变量**（ $\lambda$ ），这样会减少需要遍历的维度，进而带来效率上的提升。算法 1 对其进行了高层次的总结：

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108160249-c296.webp)

> **算法 1** ：坐标上升算法的概括性展示

但即便是这样，在更新全局参数之前，也需要先遍历所有数据点一遍（假设为单批次梯度下降）。当样本点太多时，这就会成为了一个新问题，即模型更新频率太低了。不过，该算法将参数区分为全局参数和局部参数的作法，为随机梯度下降提供了一个思路。

借鉴算法 1 的思路，可以将证据下界（ `ELBO` ）分解为两部分（采用对数形式）：**由局部参数 $\phi_i$ 参数化的逐数据点局部隐变量项** 和 **由全局参数 $\lambda$ 参数化的全局隐变量项**：

$$
\mathscr{L}\left(\lambda, \phi_{1: n}\right)=\underbrace{\mathbb{E}_{q}[\log p(\beta)-\log q(\beta \mid \lambda)]}_{\text {global contribution }}+\sum_{i=1}^{n} \underbrace{\left\{\mathbb{E}_{q}\left[\log p\left(w_{i}, z_{i} \mid \beta\right]-\log q\left(z_{i} \mid \phi_{i}\right)\right\}\right.}_{\text {per-data point contribution }}
$$

令下式为全局分布：

$$
f(\lambda):=\mathbb{E}_{q}[\log p(\beta)-\log q(\beta \mid \lambda)]
$$

且令下式为第 $i$ 个数据点的逐数据点分布。

$$
g_{i}\left(\lambda, \psi_{i}\right):=\mathbb{E}_{q}\left[\log p\left(w_{i}, z_{i} \mid \beta\right]-\log q\left(z_{i} \mid \phi_{i}\right)\right.
$$

则 `ELBO` 下界可简写为：

$$
\mathscr{L}\left(\lambda, \phi_{1: n}\right)=f(\lambda)+\sum_{i=1}^{n} \mathrm{g}_{i}\left(\lambda, \phi_{i}\right)
$$

为了优化目标，可以首先关于参数 $\phi_{1: n}$ 最大化，这将产生面向单参数的下界：

$$
\mathscr{L}(\lambda)=f(\lambda)+\sum_{i=1}^{n} \max _{\phi_{i}} \mathrm{g}_{i}\left(\lambda, \phi_{i}\right) \tag{9}
$$

令每个数据点的优化为：

$$
\phi_{i}^{*}=\arg \max _{\phi} \mathrm{g}_{i}\left(\lambda, \phi_{i}\right)
$$

则单参数下界 $\mathscr{L}(λ)$ 的梯度具有以下形式 ：

$$
\frac{\partial \mathscr{L}(\lambda)}{\partial \lambda}=\frac{\partial f(\lambda)}{\partial \lambda}+\sum_{i=1}^{n} \frac{\partial \mathrm{g}_{i}\left(\lambda, \phi_{i}^*\right)}{\partial \lambda} \tag{10}
$$

由于逐数据点项是每个数据点贡献的总和，每个数据点的导数也可以被累加起来计算每个数据点项的导数，如公式（ 10 ）的第二项所示。这使我们可以使用随机梯度算法来频繁地更新模型，以获得更好的收敛性。 一旦全局参数 $\lambda$ 被估计出来，每个 $\phi_i$ 也都可以在线估计出来。

### 4.2 参数梯度与自然梯度

坐标上升法以及上述面向随机梯度方法的改造，都解决不了另外一个问题：下界的优化是在变分参数空间内进行的，并非基于概率分布，而这会导致更新和收敛速度变慢。这是由于变分参数仅仅是为了描述`变分分布`的，下界在变分参数空间中的梯度，通常并不是概率分布空间中上升（或下降）最快的那个方向。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211114112004-358c.webp)

> 图 10：变分参数空间中的梯度与概率分布空间中的梯度

产生上述原因的根本原因在于：**变分参数空间中的欧式距离无法准确度量分布之间的距离。**

举例说明这种现象：

两组具有相同均值和方差的高斯分布对（可以想象为变分分布 $q$ 和 真实分布 $p$ ），虽然两者在变分参数（此例指均值）空间中具有相同的“距离”，但其 `KL 散度`（ 即下图中同颜色的两个高斯分布之间的重叠区域，下界会有与其相对应的反应 ）却截然不同。如果固定均值为 0 ，仅考虑方差作为变分参数时，会有类似现象产生。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108161120-f068.webp)

> 图 11：具有相同变分参数（此处为均值）的两对高斯分布，虽然在变分参数空间中，两者距离相等，但是在概率分布的相似性度量上（可直观理解为两个分布之间的重叠区域），两者并不相等。同时表明，在参数空间的欧式距离上求梯度，不能代表最速下降（或上升）方向。

那能否能够直接基于概率分布空间做优化求解呢？答案是肯定的：那就是**将自然梯度方向作为优化的梯度方向**。

> 自然梯度是 Amari 于 1998 年提出的，主要用于衡量基于统计模型（如 `KL 散度`）的目标函数。此处知识点可参见 [文献综述](https://arxiv.org/pdf/1412.1193)

无论是从 `KL 散度`的角度，还是从变分下界的角度，我们期望的目标函数 $\mathscr{L}(\lambda,\phi_{1..n})$ 都是基于概率分布的。 [文献](https://arxiv.org/pdf/1412.1193) 证明： Fisher 信息矩阵（Fisher Information Matrix, FIM）是两个概率分布之间 `KL 散度`的二阶近似，它表示了统计流形（即概率分布空间）上的局部曲率。进而推导得出，自然梯度可以定义为：

$$
\delta \theta^*==\frac{1}{\lambda} F^{-1}\nabla_\theta \mathscr{L}(\lambda,\phi_{1..n}) 
$$

式中的 $F$ 为 Fisher 信息矩阵。根据公式也可以看出，自然梯度考虑了参数空间上的曲率信息 $ \nabla_\theta \mathscr{L}(\lambda,\phi_{1..n})$。

### 4.3 随机变分推断（ `SVI` ）：自然梯度与 `SGD` 的结合

既然给出了目标函数的最速梯度方向，那么与 4.1 节的随机梯度下降相结合就成为一种非常自然的想法。大家都知道， `SGD` 是小批量梯度下降的特例，由于每次仅随机地使用一个样本（这也是取名为随机梯度下降的原因），因此会引入较大的方差，但总体趋向于最优解。以下算法 2 为**随机变分变分推断算法**， 其基本思想是：在每一轮迭代中，随机抽取一个样本数据点并计算最优局部参数，然后根据自然梯度公式更新全局参数，直至收敛。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211108163233-7872.webp)

### 4.4 随机变分推断（ `SVI` ）中的一些技巧

在随机变分推断中，随机梯度下降（ `SGD` ）的收敛速度取决于梯度估计的方差，较小的梯度噪声允许较大的学习率，并导致更快的收敛。本节介绍随机变分推断背景下的权衡技巧，例如自适应学习率和方差减少策略。其中一些方法通常适用于 `SGD`。

#### 4.4.1 自适应学习率与小批量大小

收敛速度受学习率和批量大小 [10]、[46] 的影响。基于大数定律，增加批量大小会较小随机梯度的噪声 [46]，进而允许更大的学习率。因此，为了加速学习过程，可以在给定学习率的情况下，优化调整批量大小，或者固定批量大学，优化学习率。

**（1）学习率的自适应调整**

在每次迭代中，梯度的经验方差可以用于指导学习率的自动适应（ 因为梯度噪声与学习率成反比 ）。利用此想法的优化方法包括 `RMSProp` [191]、`AdaGrad` [42]、`AdaDelta` [218] 和 `Adam` [87] 等。这些方法不是随机变分推断特有的，但在其中经常使用；更多详细信息请参阅 [53].

[157]首先在随机变分推断中引入了全局变分参数 $γ$ 的自适应学习率，其中最佳学习率被证明满足：

$$
ρ^∗_t = \frac{(γ^∗_t −γ_t )^T (γ^∗_t −γ_t )}{ (γ^∗_t − γ_t )^T (γ^∗_t −γ_t )+tr(Σ)}
$$

上面，$γ^∗_t$ 表示最优全局变分参数，$γ_t$ 表示当前估计。 $Σ$ 是这个批次中变分参数的协方差矩阵。由于 $γ^∗_t$ 是未知的，[157] 展示了如何以在线方式估计最佳学习率。

**（2）批量大小的自适应调整**

可以在保持学习率固定的同时调整批量大小，这也实现了类似效果 [10] [26]、[37]、[184]。为了减少随机梯度下降的方差，[10] 建议根据目标函数的值与其最优值成比例地选择批量大小。在实践中，估计的梯度噪声协方差和梯度大小被用于估计最佳批量大小。


#### 4.4.2 方差减少的策略

除了通过学习率和小批量大小控制优化路径之外，还可以采取一些措施来减少方差，从而实现更大的梯度步长。 随机变分推断中经常使用`方差减少`来实现更快的收敛。下面总结了一些通过`控制变量`、`非均匀采样`和其他方法实现这一目标的方法：

**（1）控制变量法**

控制变量是一个随机项，该随机项可以被添加到随机梯度中，以使梯度的期望保持不变，而方差减少 [20]。控制变量需要与随机梯度相关，并且易于计算。使用控制变量来减少方差在蒙特卡洛模拟和随机优化 [165]、[208] 中很常见。几位作者建议在随机变分推断中使用控制变量[78]、[146]、[154]、[208] 。

其中一个突出例子，是`随机方差减小梯度 (SVRG) ` [78]。在 `SVRG` 中，利用来自所有数据点获得的历史梯度来构建控制变量，并且充分利用了沿优化路径的梯度具有相关性这一特点。标准随机梯度更新 $γ_{t+1} = γ_t −ρ_t (\nabla  \hat{\mathscr{L}}(γ_t))$ 被替换为：

$$
γ_{t+1} = γ_t − ρ_t (\nabla  \hat{\mathscr{L}} (γ_t ) − \nabla  \hat{\mathscr{L}}( \tilde{γ})+\tilde{μ})
$$ 

$\hat{\mathscr{L}}$ 表示基于当前批量集索引的待估计目标（ 此处为负的 `ELBO` ），$\tilde γ$ 是每 $m$ 次迭代后 $γ$ 的快照，$\tilde μ$ 是在所有数据点上计算的批梯度，$\tilde μ = \nabla \mathscr{L}(\tilde γ)$。由于 $ − \nabla  \hat{\mathscr{L}}(\tilde γ)+ \tilde μ$ 的期望为零，因此它是一个控制变量。 

`SVRG` 需要每 $m$ 次迭代对数据集进行一次完整的遍历以计算完整的梯度，即便这种完整遍历可以被放宽到大数据集的某个非常大的批次上。对于平滑但不是强凸的目标，与 `SGD` 的 $O(1/\sqrt{T})$ 相比，`SVRG` 被证明可以实现 $O(1/T )$ 的渐近收敛率。在实践中还有许多其他控制变量 [140]、[146]、[203]。其中在 5.2 节还会介绍另外一种比较流行的控制变量类型 --- 得分函数控制变量。

**（2）非均匀采样法**

可以使用非均匀二次采样来获得具有较低梯度方差的批量，而不是以等概率对数据点进行二次采样。有几位作者建议在批量选择时采用重要性采样及其变体 [32]、[55]、[148]、[226] 。这些方法有效但并不总是实用，因为采样机制的计算复杂度与模型参数的维度有关 [47]。

另外一种替代方法旨在消除相似点的相关性并对多样化的批量进行采样。这些方法包括：`分层抽样` [225]，基于元数据或标签从预定义的子组中抽样数据；`基于聚类的抽样` [47]，使用 $k$ 均值对数据进行聚类，然后从每个聚类中按照调整后的概率抽取样本；`多样化的批量采样` [223]、[224] 使用`排斥点过程`来抑制同一小批量中具有相似特征的数据点出现的概率。

上述这些方法都已被证明可以减少方差，也可以用于学习不平衡数据。

**（3）`Rao-Blackwellization` 及其他**

已经开发了许多有助于减少随机变分推断方差的其他方法。一种流行的方法依赖于 [154] 中使用的`Rao-Blackwellization` 。 `Rao-Blackwellization 定理` 指出，如果存在可以作为条件估计的有效统计量，则条件估计具有较低的方差。受 `Rao-Blackwellization` 的启发，已经提出了`局部期望梯度方法` [194]。该方法将 `ELBO` 的梯度计算拆分为一个蒙特卡洛估计和一个精确期望，以便最佳地考虑每个隐变量维度对梯度方差的贡献。

还有其他一些随机变分推断的方法，例如， [112] 在一个批量的滑动窗口上对预期的充分统计量做平均，以获得具有较小均方误差的自然梯度。

### 4.5 其他可扩展推断方法

与利用随机优化来使收敛更快的方法相比，本节介绍了一些利用模型结构来实现相同目标的方法。特别是其中的`折叠推断`、`稀疏推断`、`并行和分布式推断` 方法。

#### 4.5.1 折叠变分推断

`折叠变分推断 (CVI)` 依赖于能够解析地积分出部分隐变量的想法 [64]、[83]、[94]、[97]、[182]、[188]、[197] 。这样的话，需要估计的参数数量会减少，推断过程通常会更快。

正是由于解析的要求，折叠推断通常限制在传统共轭指数族上，其中 `ELBO` 在边缘化期间采用解析形式获得。对于这些模型，可以在推导出 `ELBO` 之前将某些隐变量边缘化，或者在之后消除它们 [64]、[83]。几位作者提出了用于主题建模的折叠变分推断 [94]、[188] ，其中可以折叠主题比例 [188] 或主题分配 [64]。除了这些特定于模型的推导之外，[64] 统一了现有的特定于模型的 `CVI` 方法，并为共轭指数族类中的模型提供了一种通用的折叠推断方法。

`CVI` 的计算优势很大程度上取决于折叠变量的统计量。此外，折叠隐随机变量可以使其他推断技术易于处理。对于主题模型等，可以折叠离散变量，仅推断连续变量。这允许使用推断网络[122]、[180]。更一般地说，`CVI` 并不能解决所有问题。一方面，整合某些模型变量会使 `ELBO` 更紧致，因为边缘似然不必在这些变量中获得下界。另一方面，除了数学挑战之外，边缘化变量还会在变量之间引入额外的依赖关系。例如，在隐狄利克雷分配中，折叠全局变量会在赋值变量之间引入非局部依赖性，从而使分布式推断更加困难。

#### 4.5.2  稀疏变分推断

稀疏变分推断旨在通过引入了额外的低秩近似，以实现更具可扩展性的推断方法 [63]、[177]、[195]。稀疏推断既可以被解释为一种建模选择，也可以被解释为一种推断方案 [24]。读者们在`高斯过程 ( GP )` 的相关文献中经常会遇到稀疏推断方法。高斯过程的学习计算成本是 $O(M^3)$，其中 $M$ 是观测数据点的数量。高斯过程过得这种复杂度是由对 $M \times M$ 的核矩阵求逆导致的，因此极大地阻碍了高斯过程在大数据集上的应用。 

 [177] 提出稀疏高斯过程推断的想法。该工作在已知观测数据点基础上，人为地引入了 $T$ 个诱导点，并将这些诱导点解释为 “能够反映原始数据特征” 的伪输入。诱导点的数量 $T \ll M$ ，从而产生了一个更为稀疏的表示。而对于这些诱导点来说，只需要对一个 $T \times T$ 大小的矩阵求逆即可，因此计算复杂度为 $O(MT^2)$，得到极大地降低。 
 
 `[195]` 折叠了诱导点的分布；`[63]` 进一步将该工作扩展到了一个计算复杂度为 $O(T^3)$ 的随机版本；此外，稀疏诱导点使`深度高斯过程`中的推断变得易于处理 [35]。

#### 4.5.4 分布式和并行推断

变分推断可以调整到分布式计算场景，其中数据或参数的子集分布在多台机器上 `[21]、[49]、[135]、[138]、[219]`。在大规模场景中通常需要分布式推断方案，其中数据和计算分布在多台机器上。独立的隐变量模型可以简单地并行化；然而可能需要模型的特定设计（例如重参数化），以实现高效的分布式推断 `[49]`。当前的计算资源使变分推断适用于网络规模的数据分析 `[219]`。

## 5 提升通用性 --- 黑盒变分推断

### 5.1 为何要做黑盒变分推断（BBVI）？

上面章节中，我们针对特定模型做出变分推断，其中大家应该已经注意到了，在 `ELBO` 的计算表达式中，需要人为设定 $q$ ，并给出其数学期望的解析表达式（事实上，[文献](http://www.nowpublishers.com/article/Details/MAL-001) 表明，该方法只适用于**条件共轭指数族分布**）。考虑到现实世界中可能存在无数种模型，而且大部分可能是非共轭的，另外，即便符合条件共轭假设，为每一个模型设计一种变分方案显然也是不可接受的。因此，人们自然而然在思考：

**“是否存在一个不需特定于某种近似模型的通用变分推断的解决方案 ？”**

这个解决方案最好将像黑匣子一样，只需输入模型和海量数据，然后就自动输出变分分布（ 或变分参数 ）。事实表明，这是有可能的，此类推断方法被称为**黑盒变分推断（BBVI）**。

> 黑盒变分推断的概念最早出现在文献 [Ranganath et al., 2014](https://arxiv.org/pdf/1401.0118) 中和 [Sal-imans 和 Knowles，2014](https://arxiv.org/pdf/1401.1022)； [Kingma and Welling, 2014](https://arxiv.org/abs/1312.6114v10) 和 [Rezende et al., 2014](https://arxiv.org/abs/1401.4082) 提出了利用`重参数化技巧`实现反向传播和优化的方法；[Rezende and Mohamed, 2015](https://arxiv.org/abs/1505.05770) 提出了`标准化流`的黑盒变分推断方案；[Tran et al.,2016](https://arxiv.org/abs/158.06499) 提出了`高斯过程的黑盒变分推断方案`，均提升了变分推断的精度；[Alp Kucukelbir et al, 2016](https://arxiv.org/abs/1603.00788) 提出`自动微分变分推断方法（ ADVI ）`；[Yuri Burda et al., 2016](https://arxiv.org/abs/1505.00519) 在 `VAE` 基础上，提出了重要性加权变分自编码器；[J Domke and D Sheldon, 2018](https://arxiv.org/abs/1807.09034) 对其进行了泛化，提出了重要性加权变分推断。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211115171851-9244.webp)

> > 图 12： 变分推断概念图，愿景是：（1）可以轻松对任何模型进行变分推断；（2）可以利用海量数据进行推断；（3）用户只需指定模型而不需要做其他数学工作。

黑盒变分推断大致分为两种类型：

- **基于评分梯度**的黑盒变分推断

- **基于重参数化梯度**的黑盒变分推断

后者是变分自编码器 (`VAE`) 的基础。

### 5.2 使用评分梯度的 BBVI

考虑如下概率模型，其中 $\mathbf{x}$ 是观测变量， $\mathbf{z}$ 是隐变量，其`变分分布`为 $q(\mathbf{z} \mid \lambda)$ 。变分下界 (`ELBO`) 为：

$$
\mathscr{L}(\lambda) \triangleq \mathbb{E}_{q_{\lambda}(\mathbf{z})}[\log p(\mathbf{x}, \mathbf{z})-\log q(\mathbf{z} \mid \lambda)] \tag{12}
$$

`ELBO` 关于 $\lambda$ 的梯度为：

$$
\nabla_{\lambda} \mathscr{L}=\mathbb{E}_{q}\left[\nabla_{\lambda} \log q(\mathbf{z} \mid \lambda)(\log p(\mathbf{x}, \mathbf{z})-\log q(\mathbf{z} \mid \lambda))\right] \tag{13}
$$

其中 $\nabla_{\lambda} \log q(\mathbf{z} \mid \lambda)$ 被称为评分函数。

使用式（ 13） 中的评分梯度，就可以采用蒙特卡罗方法，用`变分分布`的样本来计算 `ELBO` 的含噪声无偏梯度：

$$
\nabla_\lambda \approx \frac{1}{S} \sum_{S=1}^{S} \nabla_\lambda \log q(z_S \mid \lambda)(\log p(\mathbf{x},z_S) - \log q(z_S \mid \lambda))
$$

其中 $z_S \sim q(\mathbf{z} \mid \lambda) $ 。

<br>

---
**式 (13) 的证明**

> 此处见 [参考文献](http://www.cs.columbia.edu/~blei/fogm/2018F/materials/RanganathGerrishBlei2014.pdf)
与推导出式（ 13 ）需要两个基本事实：
- $\nabla_{\lambda} q_{\lambda}(\mathbf{z})=\frac{1}{q_{\lambda}(\mathbf{z})} \nabla_{\lambda} q_{\lambda}(\mathbf{z})=q_{\lambda}(\mathbf{z}) \nabla_{\lambda} q_{\lambda}(\mathbf{z})$

- $\mathbb{E}_{q}\left[\nabla_{\lambda} \log q_{\lambda}(\mathbf{z})\right]=0$ ， 即对数似然梯度（评分函数）的期望为零。

基于这两个事实，可以推导出 `ELBO` 的评分梯度：

$$
\begin{aligned}
\nabla_{\lambda} \mathscr{L} &=\nabla_{\lambda} \int_{z}\left[q_{\lambda}(z) \log p(x, z)-q_{\lambda}(z) \log q_{\lambda}(z)\right] \mathrm{d} z \\
&=\int_{z}\left\{\log p(x, z) \nabla_{\lambda} q_{\lambda}(z)-\left[\nabla_{\lambda} q_{\lambda}(z) \log q_{\lambda}(z)+q_{\lambda}(z) \frac{1}{q_{\lambda}(z)} \nabla_{\lambda} q_{\lambda}(z)\right]\right\} \mathrm{d} z \\
&=\int_{z} \nabla_{\lambda} q_{\lambda}(z)\left[\log p(x, z)-\log q_{\lambda}(z)-1\right] \mathrm{d} z \\
&=\int_{z} q_{\lambda}(z) \nabla_{\lambda} q_{\lambda}(z)\left[\log p(x, z)-\log q_{\lambda}(z)-1\right] \mathrm{d} z \\
&=\mathbb{E}_{q_{\lambda}}\left[\nabla_{\lambda} q_{\lambda}(z)\left(\log p(x, z)-\log q_{\lambda}(z)\right)\right]-\mathbb{E}_{q_{\lambda}}\left[\nabla_{\lambda} q_{\lambda}(z)\right] \\
&=\mathbb{E}_{q_{\lambda}}\left[\nabla_{\lambda} q_{\lambda}(z)\left(\log p(x, z)-\log q_{\lambda}(z)\right)\right]
\end{aligned}
$$

---

### 5.3 使用重参数化梯度的 黑盒变分推断

依然采用上节中的模型，变分下界 `ELBO` 为（为方便重复式 （ 9 ））：

$$
\mathscr{L}(\lambda) \triangleq \mathbb{E}_{q_{\lambda}(\mathbf{z})}[\log p(\mathbf{x}, \mathbf{z})-\log q(\mathbf{z} \mid \lambda)]
$$

假设`变分分布`可以表示成如下变换：

$$
\begin{aligned}
&\epsilon \sim S(\epsilon)\\
&\mathrm{z}= t(\epsilon, \lambda) 
\end{aligned} \Leftrightarrow \quad z \sim q(\mathbf{z} \mid \lambda)
$$

例如：

$$
\begin{aligned}
&\epsilon \sim \mathcal{N}(0,1) \\
&\mathrm{z}=\mu + \epsilon \cdot \sigma
\end{aligned} \Leftrightarrow \quad \mathrm{z} \sim \mathcal{N}\left(\mu, \sigma^{2}\right)
$$

另外假设 $\log p(\mathbf{x},\mathbf{z})$ 和 $\log q(\mathbf{z})$ 关于 $\mathbf{z}$ 可微，则可以得到 `ELBO` 关于 $\lambda$ 的重参数化的梯度：

$$
\nabla_{\lambda} \mathscr{L}=\mathbb{E}_{S(\epsilon)} \left[\nabla_{\mathbf{z}} \left [ (\log p(\mathbf{x}, \mathbf{z})-\log q(\mathbf{z} )\right ] \nabla_\lambda t(\epsilon,\lambda)\right] \tag{10}
$$

进而可以使用自动微分来获取梯度，但并不是所有的分布都可以重参数化。

> 注解：
>
> (1) 此部分最经典案例见 [变分自编码器](http://www.cs.columbia.edu/~blei/fogm/2018F/materials/KingmaWelling2013.pdf)
> 
> (2) 可微性是重参数化技巧使用的重要条件，这意味着其适用范围不如评分函数方法。

### 5.4 黑盒变分推断中的方差减少策略

黑盒变分推断需要一套不同于`随机变分推断`中使用的方差减少技术。与随机变分推断的噪声来自有限数据点集的二次采样不同，黑盒变分推断的噪声来自可能具有无限支持的随机变量。在这种情况下，诸如`随机方差减少梯度 ( SVRG )` 之类的技术不再适用，因为完整梯度无法拆解成有限项的和，而且很难保存在内存当中。因此，黑盒变分推断会涉及一组不同的控制变量和其他方法，这里将简要回顾一下。

**(1) 得分函数控制变量**

黑盒变分推断中最重要的控制变量是`得分函数控制变量`，其从梯度估计器中减去得分函数的期望（蒙特卡罗法）。根据需要，`得分函数控制变量`在变分分布下的期望为零。选择权重 $w$ 的依据是其能够最小化梯度的方差。

$$
\nabla_\lambda \hat{\mathscr{L}}_{control} =\nabla_\lambda \hat{ \mathscr{L}} − \frac{w}{K} \sum_{k=1}^{K}\nabla_\lambda \text{log} q(z_k \mid \lambda) 
$$

**（2）局部期望梯度及其他**

虽然原始黑盒变分推断论文介绍了 `Rao-Blackwellization` 和`控制变量`，但 [194] 指出控制变量的良好选择可能取决于模型。因此提出了一种只考虑隐变量马尔可夫毯的局部期望梯度。 [167] 提出了一种不同的方法，它引入了`过度分散的重要性采样`。通过从属于过度分散的指数族并且在`变分分布`的尾部放置较大质量的提议分布中进行采样，可以减少梯度的方差。

## 6 提升准确性 --- 新的目标函数和变分分布

### 6.1 平均场变分的起源和局限性

变分方法在统计物理学中有着悠久的传统。平均场法最初应用于模拟自旋玻璃，这是某种类型的无序磁体，其中原子的磁自旋没有以规则模式排列 [143]。这种自旋玻璃模型的一个简单示例是 `Ising 模型` ，它是具有成对耦合晶格的二元变量模型。为了估计自旋状态的结果分布，使用更简单的、分解后的分布作为代理。这样做的目的是尽可能地近似向上或向下自旋（也称为“磁化”）的边缘概率，同时忽略自旋之间的所有相关性。给定自旋与其相邻自旋的许多相互作用，被简化为其间有效磁场（也称为平均场）的单一相互作用。这也是平均场方法名称的由来。物理学家通常将`负对数后验`表示为`能量`或`汉密尔顿函数`。这种语言也已被机器学习社区用于有向和无向概率图模型的近似推断。 

平均场方法由 Anderson 和 Peterson 于 1987 年首次在神经网络中采用 [149]，后来在机器学习社区 [79]、[143]、[172] 中广受欢迎。平均场近似的主要限制是其明确地忽略了不同变量之间的相关性，例如 `Ising 模型` 中的自旋之间的相关性。此外，[205] 表明，`变分分布`破坏的依赖关系越多，优化问题就越非凸。相反，如果`变分分布`包含更多结构，则某些局部最优就不存在了。物理界提出了许多改善平均场变分推断的举措，并由机器学习社区进一步发展 [143]、[150]、[190]。 超越自旋玻璃中的平均场理论的早期例子是 `Thouless-Anderson-Palmer (TAP) 方程` 方法 [190]，它引入了对`变分自由能`的微扰校正。

一个相关的想法依赖于幂扩展 [150]，它已被多位作者扩展并应用于机器学习模型 [80]、[142]、[145]、[158]、[185]。此外，信息几何提供了对 `MFVI` 和 `TAP 方程` [186]、[187] 之间关系的洞察。 [221] 进一步将 `TAP 方程`与`散度量`联系起来。我们将读者推荐给 [143] 以获取更多信息。接下来，我们基于 `KL 散度` 以外的散度度量来回顾 MFVI 以外的最新进展。

### 6.2 改进目标函数

 `KL 散度` 通常提供一种计算方便的方法来测量两个分布之间的距离，它导致对某些模型类的、易于处理的、解析形式的期望。然而，传统的 `KL 变分推断` (KLVI) 存在诸如低估后验方差 [128] 等问题。在某些情况下，当后验的多个峰值非常接近时， `KL 散度` 也无法打破对称性 [141]，并且它是一个相对松散的边界 [221]。出于这些缺点，我们在这里调研了许多其他的散度。

 `KL 散度` 之外的散度度量不仅在变分推断中起作用，而且在期望传播（ EP ）等相关近似推断方法中也起作用。 EP [101]、[125]、[204]、[228] 的一些最新扩展大多可以被视为替换了新散度的经典 EP [128]。这些方法有着复杂的推导和有限的可扩展性，大多数从业者会发现它们难以使用。变分推断的最新发展主要集中在黑盒方式的统一框架上，以实现可扩展性和可访问性。 黑盒变分推断使其他散度度量的应用成为可能（ 例如 $χ$ 散度 [39] ），同时保持了该方法的效率和简单性。

在本节中，我们将介绍相关的散度量并展示如何在变分推断的上下文中使用它们。如第 3.1 节所述，`KL 散度`是 `α-散度`的一种特殊形式，而 `α-散度` 是 `f-散度` 的一种特殊形式。所有上述散度都可以写成 `Stein discrepancy` 的形式。

**（1）α-散度**

从信息几何和计算的角度来看，`α-散度`是一系列具有有趣特性的散度度量 [4]、[6]。 `KL 散度` 和 `海林格（ Hellinger ）距离` 都是 `α-散度` 的特例。存在不同的`α-散度`公式 [6]、[229]，并且多种 `VI` 方法使用了不同的定义 [104]、[128]。本文采用 `Renyi` 的公式：

$$
D^R_\alpha (p||q) = \frac{1}{\alpha −1} \log \int p(x)^\alpha q(x)^{1−\alpha} \mathrm{d} x \tag{17}
$$

其中 $α > 0,\ α \neq 1$ 。根据 `α 散度` 的这个定义，较小的 $α$ 会导致更多的质量覆盖效应，而较大的 $α$ 会导致零强迫效应，这意味着`变分分布`规避了低后验概率的区域。对于 $α →1$，我们可以恢复为涉及 `KL 散度` 的标准 `VI` 。

`α-散度`最近被用于变分推断 [103]、[104]。类似于 `ELBO` 的推导，`α-散度`意味着边缘似然的边界：

$$
\begin{align*}
\mathscr{L}_{\alpha} =\log p(\boldsymbol{x})-D_{\alpha}^{R}(q(\boldsymbol{z}) \| p(\boldsymbol{z} \mid \boldsymbol{x})) \\
=\frac{1}{\alpha-1} \log \mathbb{E}_{q}\left[\left(\frac{p(\boldsymbol{z}, \boldsymbol{x})}{q(\boldsymbol{z})}\right)^{1-\alpha}\right] \tag{18}
\end{align*}
$$

对于 $α ≥0,α \neq 1$ ， $\mathscr{L}_α$ 是对数边缘似然的下界。有趣的是，式（ 18 ） 也允许 $α$ 为负值，在这种情况下它变成了一个上限。但请注意，此时 $D^R_α$ 不是散度。在 `α-散度` 的各种可能定义中，只有 `Renyi` 的定义能够产生式（ 18 ） 的边缘似然 $p(x)$ 边界。


**（2）f-散度与广义变分推断**

`α-散度` 是更广泛的 `f-散度` 的子集 [3], [33]， `f-散度` 形式如下：

$$
D_f (p||q) = \int q(x) f \left (\frac{p(x)}{q(x)}\right) \mathrm{d} x
$$


$f$ 是具有 $f(1)=0$ 性质的凸函数。例如 `KL散度` $KL(p||q)$ 用 `f-散度` 可以表示为 $f(r)=r \log(r)$ ，皮尔逊 $χ^2$ 距离也是一个 `f-散度`，其中 $f (r) = (r -1)^2$ 。

一般来说，只有特定选择的 $f$ 才会导致取决于边缘似然的边界，进而对变分推断有用。 

`[221]` 使用 `Jensen 不等式` 得到边缘似然的下界：

$$
p(\boldsymbol{x}) \geq \tilde{f}(p(\boldsymbol{x})) \geq \mathbb{E}_{q(z)}\left[\tilde{f}\left(\frac{p(\boldsymbol{x}, \boldsymbol{z})}{q(\boldsymbol{z})}\right)\right] \equiv \mathscr{L}_{\tilde{f}}
$$

上面，$\tilde{f}$ 是一个任意的凹函数，其中 $\tilde{f}(x)<$ $x$。该公式恢复了 $\tilde{f}=id$ 的真实边缘似然、$\tilde{f}=\log$ 的标准 `ELBO` 和 $\tilde{f}(x) \propto x^{(1-\alpha)}$ 的 `α-散度` 。对于 $V \equiv \log q(\boldsymbol{z})-\log p(\boldsymbol{x}, \boldsymbol{z})$，作者提出以下函数：

$$
\begin{aligned}
&\tilde{f}^{\left(V_{0}\right)}\left(e^{-V}\right) \\
&=e^{-V_{0}}\left(1+\left(V_{0}-V\right)+\frac{1}{2}\left(V_{0}-V\right)^{2}+\frac{1}{6}\left(V_{0}-V\right)^{3}\right) .
\end{aligned}
$$

上式中，$V_0$ 是一个可以优化的自由参数，它吸收了对边缘似然的边界依赖。作者表明，在 $V$ 中达到线性阶的项对应于 `KL 散度` ，而高阶项是使边界更紧致的校正项。这与早期关于 `TAP 方程` [150]、[190] 的工作有关，这些工作通常不会产生边界。

**（3）Stein Discrepancy 与变分推断**

`Stein 方法` [181] 最初被提出作为一个误差边界，用于衡量近似分布与感兴趣分布的拟合程度。`Stein 差`已适应现代变分推断[59]、[108]、[109]、[110]。在这里，我们介绍 `Stein 差`和使用它的两种变分推断方法：`Stein 变分梯度下降（Variational Gradient Descent, SVDG）` [109] 和 `运算符变分推断（ Operator VI）` [155]。这两种方法共享相同的目标，但以不同的方式实现优化。

`Stein 差` 是一个积分概率度量 [120]、[130]、[179]。特别是，[109]、[155] 使用 `Stein 差` 作为散度度量：

$$
D_{stein}(p,q) = \sup \nolimits_{f \in \mathscr{F}} \mid \mathbb{E}_{q(\boldsymbol{z})}[f (\boldsymbol{z})]−\mathbb{E}_{p(\boldsymbol{z}|\boldsymbol{x})}[f (\boldsymbol{z})] \mid ^2 \tag{20}
$$

$\mathscr{F}$ 表示一组平滑的实值函数。当 $q(\boldsymbol{z})$ 和 $p(\boldsymbol{z}|\boldsymbol{x})$ 相同时，散度为零。更一般地，$p$ 和 $q$ 越相似，差异越小。

式中的第二项涉及到难以处理的后验的期望。因此，`Stein 差` 只能在变分推断中用于第二项为零的 $\mathscr{F}$ 类函数。可以找到具有此性质的合适类，如下所示。通过在另一个函数 $\phi$ 上应用微分算子 $\mathscr{A}$ 来定义 $f$，其中 $\phi$ 仅限制为平滑：

$$
f (\boldsymbol{z}) = \mathscr A_p \phi (\boldsymbol{z}),
$$

其中 $\boldsymbol{z} ∼ p(\boldsymbol{z})$ 。运算符 $\mathscr A$ 的构造方式使得式中的第二个期望对于任意 $\phi$ 均为零；所有具有此性质的运算符都是有效的运算符 [155]。满足此要求的流行运算符是 `Stein 运算符`：

$$
 \mathscr A_p \phi (\boldsymbol{z})= \phi (\boldsymbol{z}) \nabla_{\boldsymbol{z}}  \log p(\boldsymbol{z},\boldsymbol{x})+\nabla_{\boldsymbol{z}} \phi (\boldsymbol{z}).
$$

`运算符变分推断` 和 `SVGD` [109] 都使用带`Stein 运算符` 的 `Stein 差` 来构造变分目标。这两种方法的主要区别在于优化算法。 `运算符变分推断` [155] 使用 `minmax`（GAN 风格）公式和黑盒变分推断直接优化变分目标；而 `SVGD` [109] 使用核化的 `Stein 差`。

通过对核和 $q$ 的特定选择，可以证明 `SVGD` 确定了 `KL 散度` 的最陡梯度方向上的最佳扰动 [109]。 `SVGD` 导致了一种方案，其中隐空间中的样本被顺序转换为近似后验。因此，虽然形式上不同，但该方法很容易让人想起标准化流方法 [159]。

### 6.3 改进变分分布的结构

`MFVI` 假设一个完全因式分解的`变分分布`；因此，无法捕获后验相关性。全分解变分模型的准确性有限，特别是当隐变量高度依赖时，例如在具有层次结构的模型中。本节讨论未完全分解、但包含隐变量之间依赖关系的`变分分布`。这些结构化的`变分分布`更具表现力，但通常需要更高的计算成本。

`允许结构化`变分分布`以捕获隐变量之间的依赖关系`是一种建模选择；不同依赖关系可能或多或少相关，并取决于所考虑的模型。例如，`LDA` [66] 的结构化变分推断表明维持全局结构至关重要，而 `Beta Bernoulli Process` [175] 的结构化变分推断表明维持局部结构更为重要。如下，我们回顾了分层模型的结构化推断，并讨论了时间序列的变分推断。

**（1）分层变分推断（Hierarchical VI）**

对于许多模型，通过保持隐变量之间的依赖关系可以使变分近似更具表现力，但这些依赖关系使得变分边界梯度的估计变得更加困难。`分层变分模型 (HVM)` [156] 是用于结构化`变分分布`的黑盒变分推断框架，适用于广泛的模型类别。为了捕捉隐变量之间的依赖关系，人们从`平均场变分分布` $\prod_i q(z_i; \lambda_i)$ 开始，但不是估计变分参数 $\boldsymbol{\lambda}$ ，而是在变分参数上放置一个先验 $q(\boldsymbol{\lambda} ; \boldsymbol{\theta} )$ 并将其边缘化：

$$
q(\boldsymbol{z}; \boldsymbol{\theta} ) =∫ \left(\prod_i q(z_i; \lambda_i)\right) q(\boldsymbol{\lambda} ; \boldsymbol{\theta}) \ \mathrm{d} \boldsymbol{\lambda} (21)
$$

新的`变分分布` $q(\boldsymbol{z}; \boldsymbol{\theta})$ 通过边缘化过程捕获依赖关系。通过模拟分层过程也可以从该分布中进行采样。通过熵下界和从分层模型中采样，可以使 `ELBO` 易于处理。值得注意的是，该方法可以用于开发`变分高斯过程 (VGP)` [201]，这是一种特殊的层次变分模型。 `变分高斯过程`应用高斯过程来生成变分估计，从而形成贝叶斯非参数先验。由于高斯过程可以对丰富的函数类进行建模，因此`变分高斯过程`能够自信地近似各种后验分布 [201]。


**(2) copula 分布及变分推断**

另一种在隐变量之间建立依赖关系的方法是 `copula VI` [60], [199]。 `copula VI` 没有使用完全因式分解的`变分分布`，而是采用变分族形式：

$$
q(\boldsymbol{z})=\left(\prod_i q(z_i; λ_i)\right) c \left(Q(z_1),...,Q(z_N )\right)
$$

其中 $c$ 是 `copula 分布`，它是边缘累积分布函数 $Q(z_1),...,Q(z_N)$ 上的联合分布。这种 `copula 分布`恢复了隐变量之间的依赖关系。

**（3）时间序列变分推断**

需要结构化变分近似的最重要的模型类之一是时间序列模型，重要例子包括`隐马尔可夫模型 (HMM) `[44] 和`动态主题模型` (DTM) [17] 等。这些模型在时间步长之间具有很强的依赖性，导致传统全因子平均场变分推断方法产生无法令人满意的结果。当将变分推断用于时间序列时，通常采用结构化的`变分分布`，明确捕获时间点之间的依赖关系，同时在其余变量中保持完全分解[12]、[17]、[45]、[76]。

这通常需要特定于模型的近似值。 [45]、[76] 为流行的时间序列模型导出了随机变分推断，包括 `隐马尔科夫模型（ HMM ）`、`隐式半马尔可夫模型 ( HSMM )` 和`分层狄利克雷过程-隐马尔科夫模型`。此外，[76] 为 `隐式半马尔可夫模型 ( HSMM )` 推导了一个加速的随机变分推断算法。 [11], [12] 推导出一个结构化的黑盒变分推断算法，用于非共轭的隐弥散模型。


### 6.4 其他非标准的方法

本节将介绍一些杂项方法，这些方法属于提高变分推断准确性的广泛范围，但不会被归类为新的散度度量或结构化模型。


**（1）混合分布的变分推断**


混合分布形成了一类非常灵活的分布，自 1990 年代以来一直在变分推断中使用 [74]、[79]。由于其灵活性和计算困难，混合模型的变分推断一直备受关注 [8]、[51]、[58]、[124]、[170]。为了拟合混合模型，可以使用辅助边界 [156]、定点更新 [170]，或强制执行额外的假设，例如使用统一权重 [51]。

受提升方法的启发，最近提出的方法以连续方式拟合混合成分[58]，[124]。在这里，`提升变分推断（Boosting VI）`和`变分提升（variational boosting）`[58]、[124] 一次添加一个组件，同时固定先前已拟合的组件，通过迭代来改进近似后验。在另一种方法中，[8] 利用强化学习文献中的随机策略搜索方法来拟合高斯混合模型。

**（2）利用随机梯度下降的变分推断**

在某些情况下，概率模型的负对数后验上的随机梯度下降可以被视为一种隐式变分推断算法。在这里，我们考虑具有恒定学习率[113]、[114] 和提前停止 [43] 的随机梯度下降（ `SGD` ）。

`恒定 SGD` 可以看作是收敛到平稳分布的马尔可夫链；因此，它类似于朗之万动力学 ( Langevin dynamics ) [214]。平稳分布的方差由学习率控制。 [113] 表明可以调整学习率以最小化所得平稳分布和贝叶斯后验之间的 `KL 散度` 。此外，[113] 推导出了一个最佳学习率的公式，这些公式结合了 `AdaGrad` [42] 及相关成.

[114] 中介绍了包括`动量`和`迭代平均`的`泛化 SGD`。相比之下，[43] 将 `SGD` 解释为非参数变分推断方案。该论文提出了一种跟踪隐式变分目标熵变化的、基于 `Hessian 估计`的新方法，因此作者考虑从非平稳分布中抽样。

**（3）对异常值和局部最优的鲁棒性**

由于 `ELBO` 是一个非凸目标，变分推断受益于高级优化算法以摆脱局部最优解。其中，`变分退火法` [115] 在变分推断中采用确定性退火 [136]、[164]，使冷却方案具有自适应性和数据相关性。

可以在全局或局部范围内定义温度，其中局部温度特定于局部的若干数据点。模型下具有较小似然的若干数据点（ 例如异常值 ）会自动被分配高温，从而减少了其对全局变分参数的影响，使推断算法对局部最优更鲁棒。变分退火法也可以解释为数据重新加权 [212]，权重是逆温度。在这种情况下，为异常值分配了较低的权重。

其他使变分推断更稳健的方法包括`信任区域法` [189]，它使用 `KL 散度` 来调整学习进度并避免局部最优；另外还有`种群变分推断`[92]，它通过对自举数据样本的变分后验进行平均，得到更为鲁棒的建模性能。

## 7 摊销式变分推断与深度学习

考虑图 2 的设置，其中每个数据点 $x_i$ 由其具有变分参数 $\xi_i$ 的隐变量 $z_i$ 制。传统的变分推断需要为每个数据点 $x_i$ 优化 $\xi_i$，这在计算上过于昂贵，特别是当这种优化嵌入到全局参数的更新循环时。

『摊销推断』背后的基本思想是使用强大的预测器根据 $x_i$ 的特征来预测最优 $z_i$，即 $z_i = f(x_i)$ 。这样，局部变分参数就被数据的函数替换，而函数中的参数在所有数据点之间共享，即推断被摊销了。我们在 7.1 节详细介绍了这种方法背后的主要思想，并在 7.2 和 7.3 节中展示了如何以变分自编码器的形式应用摊销变分推断。

### 7.1 摊销变分推断（Amortized Variational Families） 

术语『摊销推断』指利用来自过去计算的推断来支持未来的计算 `[36]、[50]`；对于变分推断，摊销推断通常是指对局部变量的推断（ 注：此处有些拗口，其实就是利用已有观测数据集来构建 “预测变量到局部隐变量” 的摊销模型，然后用于预测新数据点对应的局部隐变量 ）。

与前述各种推断方法中为每个数据点对应的隐变量设置单独的变分参数（如图 2a 所示）不同，摊销变分推断假设局部隐变量的变分参数可以通过数据的某个函数进行预测（ 该函数被 $\phi$ 参数化，所有局部隐变量共享该参数，如图 2b 所示 ）。因此，一旦估计了该函数（ 即确定了 $\phi$ ），就可以通过向其传递新数据点来获取隐变量。当该函数为一个深度神经网络时，也被称为推断网络。因此，推断网络形式的摊销变分推断将“概率建模”与“深度学习”的表示能力优雅地结合到了一起。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211223113610-8a74.webp)

> 图 2. 随机变分推断的概率图表示 (a)，和变分自编码器 (b)，其中虚线线条表示对变分的近似

摊销推断方法已应用于`深度高斯过程 (DGP)` `[35]`。 该模型中的推断难以处理，作者就将平均场变分推断与诱导点一起应用（参见 `[35]` 第 3.3 节）。进一步地，`[34]` 建议将这些隐变量估计为推断网络的函数，而不是单独估计这些隐变量，从而允许深度高斯过程扩展到更大的数据集并加速收敛。此外，还可以通过将摊销误差反馈到推断模型中来迭代摊销 `[116]`。

### 7.2 变分自编码器 （`VAE`）

摊销变分推断已成为`深度隐高斯模型 (DLGM)` 中的流行推断工具。这产生了`变分自编码器 ( VAE )` 的概念。变分自编码器由两个研究小组独立提出 [85]、[160]，其应用比较广泛，不仅限用于深度隐高斯模型，但为简单起见，本文将讨论限制在这个模型类上。

**（1）生成式模型**

深度隐高斯模型的概率图模型如图 2b 所示。该模型采用多元正态先验，我们从中抽取一个隐变量 $z$ ，
$$
p(z) = \mathcal N (0,I)
$$

更一般地，这可能是依赖于附加参数 $θ$ 的任意先验 $p_θ (z)$ 。模型的似然为：

$$
p_θ (\boldsymbol{x}|\boldsymbol{z}) = \prod_{i=1}^{N}\mathcal{N} (x_i; μ(z_i),σ^2(z_i)I)
$$

最重要的是，似然通过两个非线性函数 $μ(·)$ 和 $σ (·)$ 而依赖于 $\boldsymbol{z}$。这些非线性函数通常是神经网络，它将隐变量作为输入并以非线性方式进行转换。然后从 『以转换后的变量 $μ(z_i)$ 为中心的正态分布中』抽取数据。参数 $θ$ 包含网络 $μ(·)$ 和 $σ (·)$ 中的参数。

深度隐高斯模型是高度灵活的密度估计器。存在许多特定于其他数据类型的改进版本。例如，对于二进制数据，高斯似然可以替换为伯努利似然。下面我们回顾如何将摊销推断应用于该模型类。

**（2）变分自编码器（ `VAE` ）**

变分自编码器（ `VAE` ）是指使用推断网络训练的深层隐高斯模型。 `VAE` 使用两组深度神经网络：一是上述的自顶向下的生成模型，从隐变量 $\boldsymbol{z}$ 映射到数据 $\boldsymbol{x}$ ； 二是获得近似后验 $p(\boldsymbol{z}|\boldsymbol{x})$ 的自底向上的推断模型。通常，对应的神经网络被称为`生成网络`和`推断网络`，从机器学习视角看，也被称为`解码器`和`编码器`网络。 

为了近似后验，`VAE` 采用`摊销平均场变分分布`：

$$
q_\phi (\boldsymbol{z}|\boldsymbol{x}) = \prod_{i=1}^N q_\phi (z_i|x_i)
$$

$x_i$ 条件化表明与每个数据点相关的局部变分参数被数据的某个函数所取代。

这种`摊销变分分布`的一种常见选择是使用高斯模型：

$$
q_\phi (z_i|x_i) = \mathcal{N} (z_i|μ(x_i),σ^2(x_i)I))
$$

与生成模型类似，`变分分布`采用数据的非线性映射 $μ(x_i)$ 和 $σ(x_i)$ 来预测 $z_i$ 的近似后验分布。参数 $\phi$ 涵盖了推断网络中的参数.

[85]、[160] 的主要贡献是为深度隐变量模型推导出了一个可扩展且高效的训练方案。在优化过程中，推断网络和生成网络共同训练以优化 `ELBO`。

训练该模型的关键是`重新参数化技巧`。我们关注来自单个数据点 $x_i$ 的 `ELBO` 贡献。首先，从噪声分布中抽取 `L` 个样本 $ε_{(l,i)} ∼ p(ε)$。我们还使用了重参数化函数 $g_\phi$ ，使得 $z_{(i,l)} = g_\phi (ε_{(l,i)},x_i)$ 实现来自近似后验 $q_\phi (z_i|x_i)$ 的样本。对于上式，最常见的重参数化函数采用 $z_{(i,l)} = μ(x_i​​) + σ (x_i) ∗ε_{(i,l)}$ 的形式，其中 $μ(·)$ 和 $σ (·)$ 由 $\phi$ 参数化。获得变分自编码器的 `ELBO` 无偏蒙特卡罗估计量：

$$
\hat{\mathscr{L}} (\theta ,\phi ,x_i) = − D_{KL}(q_\phi (z_i|x_i)||p_\theta (z_i)) + \frac 1L \sum_{l=1}^L \log p_\theta(x_i|μ(x_i)+σ(x_i)∗ε_{(i,l)}) 
$$

`ELBO` 的这种随机估计随后可以根据 $θ$ 和 $\phi$ 进行微分，以获得梯度的估计。重新参数化技巧还意味着梯度方差受常数限制 [160]。然而，这种方法的缺点是我们需要近似后验是可重新参数化的。 


> **编码器-解码器的概率视角**
>
> 术语`变分自编码器`源于这样一个事实，即生成和推断网络的联合训练类似于自编码器（一种无监督的确定性模型）的结构。自编码器是经过训练的深度神经网络，能够尽可能地重建其输入。重要的是，自编码器中涉及的神经网络具有沙漏结构，这意味着有存在少量单元的隐藏层用于阻止神经网络学习琐碎的映射。这个“瓶颈”迫使网络学习有用且紧凑的数据表示。相比之下，`VAE` 是一个概率模型，但它们与经典自编码器有密切的对应关系。事实证明，`VAE` 的隐藏变量可以被认为是自编码器的 “瓶颈” 中的中间表示。在 `VAE` 训练期间，将噪声注入该中间层，具有正则化作用。此外，先验和近似后验之间的 `KL 散度`项`迫使 `VAE` 的隐表示接近先验，导致隐空间中的分布更均匀，并更好地泛化到未观测数据。当噪声方差减少到零并省略先验项时，`VAE` 就是经典的自编码器。

### 7.3 更灵活的 `VAE` 


自从提出 `VAE` 以来，提出了越来越多的扩展。该主题本身就可以写一篇评论文章，因此，本文仅总结了其中一些重要的扩展，并且更加强调推断过程。我们先讨论修改变分分布 $q_\phi$ 和模型 $p_θ$ 的扩展，最后讨论当一些隐单元的后验在优化过程中与先验保持接近时的死亡单元问题。

#### 7.3.1 更灵活的`变分分布` $q_\phi$ 

包括 `VAE` 在内的传统变分推断依赖于参数化的推断模型。近似后验 $q_\phi$ 可以是显式的参数分布，例如高斯分布或离散分布 [163]。我们也可以使用更灵活的分布，例如通过对简单参数分布做转换。在这里，我们回顾了`具有隐式分布的 `VAE`、`标准化流`和`重要性加权 `VAE`。使用更灵活的`变分分布`不仅可以减少近似误差，还可以减少摊销误差，即由于用参数化的函数替换局部变分参数而引入的误差 [30]。

**（1）隐式分布**

隐式分布可以在变分推断中使用，因为封闭形式的密度函数是对推断模型没有严格要求；我们所需要的只是能够从中采样。如下详述，它们的重新参数化梯度仍然可以计算。除了标准的重新参数化方法之外，还必须估计对梯度的熵贡献。变分推断的隐式分布是一个活跃的研究领域 [72]、[81]、[102]、[105]、[107]、[119]、[129]、[196]、[210]。

变分推断需要计算对数密度比 $\log p(\boldsymbol{z})−\log q_\phi (\boldsymbol{z}|\boldsymbol{x})$ 。当 $q$ 为隐式时，标准训练程序面临对数密度比难以处理的问题。在这种情况下，可以使用`生成对抗网络 (GAN)` [54] 风格的鉴别器 $T$ ，以从`变分分布`中判别出先验，$T (\boldsymbol{x},\boldsymbol{z}) = \log q_\phi (\boldsymbol{z}|\boldsymbol{x})−\log p(\boldsymbol{z})$ [ 102]、[119]。这个公式非常通用，可以与其他想法结合，例如分层结构 [202]、[217]。

**（2）标准化流方法**

`标准化流` [29]、[40]、[41]、[84]、[159] 提出了另一种灵活变分分布的方法。标准化流背后的主要思想是通过一系列连续的可逆变换将简单（例如平均场）近似后验 $q(\boldsymbol{z})$ 变换为更具表现力的分布。

为此，首先抽取一个随机变量 $z ∼q(\boldsymbol{z})$ ，并使用可逆的平滑函数 $f$ 对其进行变换。设 $z′ = f (z)$。那么，新的分布是：

$$
q(z^′) = q(z)|\frac{\partial f^{−1}}{ ∂ z^′} |= q(z)|\frac{\partial f}{∂ z^′}|^{−1}
$$

我们有必要计算行列式，因为变分方法要求估计变换分布的熵。通过选择变换函数 $f$ 使得 $|\frac{\partial f}{\partial z'}|$ 更容易计算，这种标准化流程构成了一种从简单分布生成多峰分布的有效方法。作为其变体，已经提出了`线性时间变换流`、`Langevin 流` 和 `Hamiltonian 流` [159] 以及`逆自回归流` [84] 和`自回归流` [29]。

`标准化流`和前面提到的`隐式分布`采取了 “使用转换将简单分布转换为复杂分布” 的共同想法，它们之间一个关键区别在于，由于采用了可逆变换函数，标准化流允许估计 $q(z)$ 的密度。

**（3）重要性加权 `VAE`**

利用灵活变分分布的最后一种方法是`重要性加权变分自编码器 (IWAE)`，它最初被提议用于收紧变分边界 [25]，并且可以重新解释为从更灵活的分布 [31] 中采样。 `IWAE` 需要来自近似后验的 `L` 个样本，这些样本按比率进行加权：

$$
\hat w_l = \frac{w_l}{\sum^L_{l=1} w_l} \ , \ \ w_l = p_\theta (x_i,z_{(i,l)})
q_\phi (z_{(i,l)}|x_i)
$$


作者表明，评估的样本 `L` 越多，变分边界变得越紧，这意味着在 $L →∞$ 时接近真实对数似然。对 `IWAE` 的重新解释表明，它们与 `VAE` 相同，但从更具表现力的分布中采样，该分布在 $L →∞$ [31] 时逐点收敛到真实后验。由于 `IWAE` 引入了有偏估计器，因此隐地可以采取额外步骤来获得更好的`方差-偏差权衡` [139]、[152]、[153] 。

#### 7.3.2 先验 $p_θ$ 的选择

建模选择会影响深度隐高斯模型的性能。特别是改进 `VAE` 中的先验模型可以带来更多可解释的拟合和更好的模型性能。 [77] 提出了一种将结构化先验用于 `VAE` 的方法，结合了传统概率图模型和推断网络的优点。这些混合模型通过使用识别模型来学习共轭分布的变分参数，克服了非共轭先验和似然的难点。这允许人们在保持共轭的同时近似后验。当编码器输出自然参数的估计值时，依赖共轭的`消息传递`被应用于执行剩余的推断。

还有其他方法解决了标准 `VAE` 的缺点。由于标准 `VAE` 假设似然在维度上可分解，因此可能产生一个糟糕的近似，例如，对于结构化输出模型效果更好的图像。 `Deep Recurrent Attentive Writer` [56] 依赖于一种循环结构，该结构逐渐构建观测结果，同时自动关注感兴趣的区域。相比之下，`PixelVAE` [57] 则通过对图像内像素之间的依赖关系建模来解决这个问题，该方法使用了一个条件模型的分解式 $p_\theta (x_i|z_i) = \prod_j p_\theta (x^j_i \mid x^1_i ,...x^{j−1}_
i,z_i)$ ，其中 $x^j_i$ 表示观测 $i$ 的第 $j$ 个维度。维度以顺序方式生成，这说明了图像像素内的局部依赖性。

建模选择带来的表现力是需要代价的。如果解码器太强，推断过程可能无法学习信息丰富的后验 [29]。该问题被称为`僵尸单元问题`。

#### 7.3.3 僵尸单元的问题

某些建模选择和参数配置给 `VAE` 训练带来了问题，例如学习数据的良好低维表示容易失败。一个突出的问题被称为`僵尸单元问题`。造成这种现象的主要有两个原因：`解码器过于强大`以及 `KL 散度项`。

在某些情况下，解码器的表达能力非常强，以至于 $\boldsymbol{z}$ 变量的某些维度被忽略了，即它可能独立于 $z$ 对 $p_\theta (x|z)$ 进行建模。此时真实后验变成了先验 [29]，因此变分后验试图匹配先验以满足等式中的 `KL 散度` 。 `有损变分自编码器` [29] 通过在部分输入信息上调节每个输出维度的解码分布来规避此问题。例如，在图像案例中，给定像素的似然仅取决于周围像素的值和全局隐状态，这迫使分布编码了隐变量中包含的全局信息。

 `KL 散度` 对 `VAE` 损失的贡献可能会加剧这个问题。要了解原因，可以将 `ELBO` 重写为两个 `KL 散度` 之和 $\hat{\mathscr L}(\theta ,\phi ,x_i) = −D_{KL}(q_\phi (z|x_i)||p_\theta (z)) −D_{KL}(p(x_i)|| p_\theta (x_i|z)) + C$ 。如果模型表达能力足够强，则模型能够将第二项置为零（与 $\boldsymbol{z}$ 的值无关）。此时，为了同时满足第一项，推断模型将其概率质量与先验 [227] 相匹配，从而未能学习到数据的有用表示。即使解码器不强，在优化早期阶段可能会出现僵尸单元问题，此时近似后验尚未携带有关数据的相关信息 [19]。当 $z$ 的维数很高时，这个问题更加严重。这种情况下，单元会朝着先验方向正则化，并且可能无法在优化后期阶段重新激活 [178]。为了抵消 `KL 约束`的早期影响，可以在训练期间对 `KL 散度项`应用退火方案 [178]。

## 8 讨论

下面是一些活跃的研究方向和开放性问题：

### 8.1 变分推断理论方面

尽管在建模和推断方面取得了进展，但很少有作者讨论变分推断 的理论方面 [95]、[133]、[213]。一个重要方向是量化`变分分布`替换真实后验时的近似误差 [133]。与此相关的一个问题是预测误差，例如，使用变分推断近似来做贝叶斯预测分布的边缘化计算。我们还推测变分推断理论可以从与信息论的联系中受益。这已经在 [186]、[187] 中举例说明。信息论还激发了新模型和推断方案的发展 [2]、[13]、[193]。例如，信息瓶颈 [193] 最近推动了深度变分信息瓶颈 [2]。我们期望融合这两条研究线会产生更多有趣的结果。

### 8.2 变分推断和深度学习

尽管最近在各领域取得了成功，但深度学习仍然缺乏原则性的不确定性估计、缺乏其特征表示的可解释性，并且难以包含先验知识。贝叶斯方法（例如贝叶斯神经网络 [137] 和变分自编码器）正在改进这些方面。最近的工作旨在使用可解释性概率模型作为 `VAE` 的先验 [38]、[77]、[91]、[168]。在此类模型中，变分推断是必不可少的组成部分。在贝叶斯深度架构中，如何使变分推断计算更为高效且易于实现，正在成为一个重要研究方向 [48]

### 8.3 变分推断和策略梯度

策略梯度估计对于`强化学习 (RL)`[183]​​ 和随机控制很重要。这些应用中的技术挑战与变分推断非常相似 [98]、[99]、[110]、[173]、[211] 。例如，`SVGD` 已作为 `Stein Policy 梯度`被应用于 RL 设置 [110]。 变分推断在强化学习中的应用目前是一个活跃的研究领域。

### 8.4 自动变分推断

概率编程允许从业者快速实现和修改模型，而不必担心推断问题。用户只需要指定模型，推断引擎就会自动进行推断。流行的概率编程工具包括但不限于：Stan[28]，涵盖了大量的高级变分推断和 `MCMC` 推断方法； Net[126] 基于变分消息传递和期望传播；Automatic Statistician[52] 和 Anglican[198] 主要依靠采样方法；Ed-ward[200] 支持黑盒变分推断和蒙特卡洛采样 ； Zhusuan[176] 的特点是用于贝叶斯深度学习的变分推断。这些工具的长期目标是改变概率建模的研究方法，使用户能够快速修改和改进模型，并使其他受众可以访问它们。

尽管目前努力使从业者更容易使用变分推断，但对于非专家来说，其使用仍然不简单。例如，人工识别后验的对称性并打破这些对称性是推断网络所必需的。此外，诸如控制变量等减少方差的方法可以极大地加速收敛，但需要模型进行特定设计才能获得最佳性能。在撰写本文时，当前的概率编程工具箱尚未解决此类问题。我们相信这些方向对于推进概率建模在科学和技术中的影响非常重要。

## 9 总结

- 概率机器学习是将领域知识与数据联系起来的机器学习方法
- 它提供了一种用于分析数据的计算方法
- 概率机器学习的最终目标是形成一种具有表现力、可扩展、易于开发的方法
- 后验推断的关键是算法问题
- 变分推断为后验推断提供了可扩展和通用方法
- 平均长近似和坐标上升方法是最为基础的变分推断方法
- 随机变分推断将变分推断扩展到海量数据
- 黑盒变分推断将变分推断泛化到多种模型


## 参考.

[1] S. Ahn, A. Korattikara, and M. Welling. Bayesian posterior sampling via stochastic gradient fisher scoring. In ICML, 201.

[2] A. Alemi, I. Fischer, J. Dillon, and K. Murphy. Deep variational information bottleneck. In ICLR, 201.

[3] S. M. Ali and S. D. Silvey. A general class of coefficients of divergence of one distribution from another. Journal of the Royal Statistical Society., 196.

[4] S.I. Amari. Differential-geometrical methods in statistics. Springer, 198.

[5] S.I. Amari. Natural gradient works efficiently in learning. Neural computation, 10(2), 199.

[6] S.I. Amari. α-divergence is unique, belonging to both f - divergence and bregman divergence classes. IEEE Trans- actions on Information Theory, 55(11), 200.

[7] E. Angelino, M. J. Johnson, and R. P. Adams. Patterns of scalable bayesian inference. Foundations and Trends R©in Machine Learning, 9(2-3), 201.

[8] O. Arenz, M.J. Zhong, and G. Neumann. Efficient gradient-free variational inference using policy search. In ICML, 201.

[9] A. Azevedo-Filho and R.D. Shachter. Laplace’s method approximations for probabilistic inferencein belief networks with continuous variables. In UAI, 199.

[10] L. Balles, J. Romero, and P. Hennig. Coupling adaptive batch sizes with learning rates. In UAI, 201.

[11] R. Bamler and S. Mandt. Dynamic word embeddings. In ICML, 201.

[12] R. Bamler and S. Mandt. Structured black box variational inference for latent time series models. In ICML WS, 201.

[13] D. Barber and F. Agakov. The IM algorithm: a variational approach to information maximization. In NIPS, 200.

[14] C. M. Bishop. Pattern recognition and machine learning. springer, 200.

[15] D. M. Blei, A. Kucukelbir, and J. D. McAuliffe. Variational inference: A review for statisticians. 201.

[16] D. M. Blei and J. D. Lafferty. Correlated topic models. In NIPS, volume 18, 200.

[17] D. M. Blei and J. D. Lafferty. Dynamic topic models. In ICML, 200.

[18] L. Bottou. Large-scale machine learning with stochastic gradient descent. Springer, 201.

[19] S. R. Bowman, L. Vilnis, O. Vinyals, A. M. Dai, R. J  ́ozefowicz, and S. Bengio. Generating sentences from a continuous space. In CoNLL.

[20] P.P. Boyle. Options: A monte carlo approach. Journal of financial economics, 4(3):323–338, 197.

[21] T. Broderick, N. Boyd, A. Wibisono, A. C. Wilson, and M. I. Jordan. Streaming variational bayes. In NIPS, 2013.

[22] S. Brooks, A. Gelman, G. Jones, and X.L Meng. Handbook of markov chain monte carlo. CRC press, 2011.

[23] A. Buchholz, F. Wenzel, and S. Mandt. Quasi-Monte Carlo Variational Inference. In ICML, 2018.

[24] T. D. Bui, J. Yan, and R. E. Turner. A unifying framework for sparse gaussian process approximation using power expectation propagation. arXiv:1605.07066, 2016.

[25] Y. Burda, R. Grosse, and R. Salakhutdinov. Importance weighted autoencoders. arXiv:1509.00519, 2015.

[26] R.H. Byrd, G.M. Chin, J. Nocedal, and Y.C. Wu. Soamploample size selection in optimization methods for machine learning. Mathematical programming, 134(1), 2012.

[27] L. Le Cam. Asymptotic methods in statistical decision theory. Springer Science & Business Media, 2012.

[28] B. Carpenter, D. Lee, M. A. Brubaker, A. Riddell, A. Gelman, B. Goodrich, J. Guo, M. Hoffman, M. Betancourt, and P Li. Stan: A probabilistic programming language. Journal of Statistical Software, 2016.

[29] X. Chen, D. P. Kingma, T. Salimans, Y. Dua, P. Dhariwal, J. Schulman, I. Sutskever, and P. Abbeel. Variational lossy autoencoder. In ICLR, 2017.

[30] C. Cremer, X Li, and D. Duvenaud. Inference suboptimality in variational autoencoders. In ICML, 2018.

[31] C. Cremer, Q. Morris, and D. Duvenaud. Reinterpreting importance-weighted autoencoders. In ICLR WS, 2017.

[32] D. Csiba and P. Richt ́arik. Importance sampling for minibatches. arXiv:1602.02283, 2016.

[33] I. Csisz ́ar. Eine informations theoretische ungleichung und ihre anwendung auf beweis der ergodizitaet von markoff18 schen ketten. Magyer Tud. Akad. Mat. Kutato Int. Koezl., 8, 1964.

[34] Z.W. Dai, A. Damianou, J. Gonz ́alez, and N. Lawrence. Variational auto-encoded deep Gaussian processes. In ICLR, 2016.

[35] A. Damianou and N. Lawrence. Deep gaussian processes. In AISTATS, 2013.

[36] P. Dayan, G. E. Hinton, R. M. Neal, and R. S. Zemel. The helmholtz machine. Neural computation, 7(5), 1995.

[37] S. De, A. Yadav, D. Jacobs, and T. Goldstein. Automated inference using adaptive batch sizes. In AISTATS, 2017.

[38] Z.W. Deng, R. Navarathna, P. Carr, S. Mandt, Y.S. Yue, I. Matthews, and G. Mori. Factorized variational autoencoders for modeling audience reactions to movies. In CVPR, 2017.

[39] A. B. Dieng, D. Tran, R. Ranganath, J. Paisley, and D. M. Blei. Variational inference via chi-upper bound minimization. In NIPS, 2017.

[40] L. Dinh, D. Krueger, and Y. Bengio. NICE: non-linear independent components estimation. arXiv:1410.8516, 2014.

[41] L. Dinh, J. Sohl-Dickstein, and S. Bengio. Density estimation using real nvp. In ICLR, 2017.

[42] J. Duchi, E. Hazan, and Y. Singer. Adaptive subgradient methods for online learning and stochastic optimization. JMLR, 12, 2011.

[43] D. Duvenaud, D. Maclaurin, and R. Adams. Early stopping as nonparametric variational inference. In AISTATS, 2016.

[44] S. R. Eddy. Hidden Markov Models. Current opinion in structural biology, 6(3), 1996.

[45] N. Foti, J. Xu, D. Laird, and E. Fox. Stochastic variational inference for hidden Markov models. In NIPS, 2014.

[46] M. P. Friedlander and M. Schmidt. Hybrid deterministicstochastic methods for data fitting. SIAM Journal on Scientific Computing, 34(3), 2012.

[47] T.F Fu and Z.H. Zhang. CPSG-MCMC: Clustering-based preprocessing method for stochastic gradient mcmc. In AISTATS, 2017.

[48] Y. Gal. Uncertainty in deep learning. PhD thesis, PhD thesis, University of Cambridge, 2016.

[49] Y. Gal, M. van der Wilk, and C. Rasmussen. Distributed variational inference in sparse gaussian process regression and latent variable models. In NIPS, 2014.

[50] S. Gershman and N. Goodman. Amortized inference in probabilistic reasoning. In CogSci, volume 36, 2014.

[51] S. J. Gershman, M. D. Hoffman, and D. M. Blei. Nonparametric variational inference. In ICML, 2012.

[52] Z. Ghahramani. Probabilistic Machine Learning and Artificial Intelligence. Nature, 521(7553), 2015.

[53] I. Goodfellow, Y. Bengio, and A. Courville. Deep Learning. MIT Press, 2016.

[54] I. Goodfellow, J. Pouget-Abadie, M. Mirza, B. Xuand D. Warde-Farley, S. Ozair, A. Courville, and Y. Bengio. Generative adversarial nets. In NIPS, 2014.

[55] P. K. Gopalan, S. Gerrish, M. Freedman, D. M. Blei, and D. M. Mimno. Scalable inference of overlapping communities. In NIPS, 2012.

[56] K. Gregor, I. Danihelka, A. Graves, D. J. Rezende, and D. Wierstra. Draw: A recurrent neural network for image generation. In ICML, 2015.

[57] I. Gulrajani, K. Kumar, F. Ahmed, A. A. Taiga, F. V. Visin, D. Vazquez, and A. Courville. PixelVAE: A latent variable model for natural images. 2017.

[58] F.J. Guo, X.Y. Wang, K. Fan, T. Broderick, and D. B. Dunson. Boosting variational inference. arXiv:1611.05559, 2016.

[59] J. Han and Q. Liu. Stein variational adaptive importance sampling. In UAI, 2017.

[60] S. Han, X. Liao, D. Dunson, and L. Carin. Variational gaussian copula inference. 2016.

[61] G. Heinrich. Parameter estimation for text analysis, 2008.

[62] P. Hennig. Approximate inference in graphical models. PhD thesis, University of Cambridge, 2011.

[63] J. Hensman, N. Fusi, and N. D. Lawrence. Gaussian processes for big data. In UAI, 2013.

[64] J Hensman, M Rattray, and N. D Lawrence. Fast variational inference in the conjugate exponential family. In NIPS, 2012.

[65] M. Hoffman, F. R. Bach, and D. M. Blei. Online learning for Latent Dirichlet Allocation. In NIPS, 2010.

[66] M. D. Hoffman and D. M. Blei. Structured stochastic variational inference. In AISTATS, 2015.

[67] M D. Hoffman, D. M. Blei, C. Wang, and J. Paisley. Stochastic variational inference. JMLR, 14(1), 2013.

[68] R. V. Hogg and A. T. Craig. Introduction to mathematical statistics.(5th edition). Upper Saddle River, New Jersey: Prentice Hall, 1995.

[69] A. Honkela, T. Raiko, M. Kuusela, M. Tornio, and J. Karhunen. Approximate riemannian conjugate gradient learning for fixed-form variational bayes. JMLR, 11, 2010.

[70] A. Honkela, M. Tornio, T. Raiko, and J. Karhunen. Natural conjugate gradient in variational inference. In ICONIP, 2007.

[71] A. Honkela and H. Valpola. Online variational bayesian learning. 2003.

[72] F. Husz ́ar. Variational inference using implicit distributions. arXiv:1702.08235, 2017.

[73] T. S. Jaakkol, L. K. Saul, and M. I. Jordan. Fast learning by bounding likelihoods in sigmoid type belief networks. NIPS, 1996.

[74] T. S. Jaakkola and M. I. Jordan. Improving the mean field approximation via the use of mixture distributions. NATO ASI series D behaviroural and social sciences, 89, 1998.

[75] E. Jang, S.X. Gu, and B. Poole. Categorical reparameterization with gumbel-softmax. In ICLR, 2017.

[76] M. Johnson and A. Willsky. Stochastic variational inference for bayesian time series models. In ICML, 2014.

[77] M. J. Johnson, D. Duvenaud, A. B. Wiltschko, S. R. Datta, and R. P. Adams. Structured VAEs: Composing probabilistic graphical models and variational autoencoders. In NIPS, 2016.

[78] R. Johnson and T. Zhang. Accelerating stochastic gradient descent using predictive variance reduction. In NIPS, 2013.

[79] M. I. Jordan, Z. Ghahramani, T. S. Jaakkola, and L. K. Saul. An introduction to variational methods for graphical models. Machine learning, 37(2), 1999.

[80] H. J. Kappen and W. Wiegerinck. Second order approximations for probability models. In NIPS, 2001.

[81] T. Karaletsos. Adversarial message passing for graphical models. In NIPS WS. 2016.

[82] M. E. Khan, P. Baqu ́e, F. Fleuret, and P. Fua. Kullback-Leibler Proximal Variational Inference. In NIPS, 2015.

[83] N. King and N. Lawrence. Fast variational inference for gaussian process models through KL-correction. In ECML, 2006.

[84] D. P. Kingma, T. Salimans, R. Jozefowicz, X. Chen, I. Sutskever, and M. Welling. Improving variational autoencoders with inverse autoregressive flow. In NIPS, 2016.

[85] D. P. Kingma and M. Welling. Auto-encoding variational bayes. In ICLR, 2014.

[86] D. P. Kingma and M. Welling. Stochastic gradient vb and the variational auto-encoder. In ICLR, 2014.

[87] D.P. Kingma and J. Ba. Adam: A method for stochastic optimization. In ICLR, 2015.

[88] D. A. Knowles and T. P. Minka. Non-conjugate variational message passing for multinomial and binary regression. In NIPS, 2011.

[89] D.A. Knowles. Stochastic gradient variational Bayes for gamma approximating distributions. arXiv, page 1509.01631, 2015. 1.

[90] A. Korattikara, V. Rathod, K. Murphy, and M. Welling. Bayesian Dark Knowledge. arXiv:1506.04416, 2015.

[91] R. G. Krishnan, U. Shalit, and D. Sontag. Deep kalman filters. In NIPS WS, 2015.

[92] A. Kucukelbir and D. M. Blei. Population empirical bayes. In UAI, 2015.

[93] S. Kullback and R. A. Leibler. On information and sufficiency. The annals of mathematical statistics, 22(1), 1951.

[94] K. Kurihara, M. Welling, and Y. W. Teh. Collapsed variational dirichlet process mixture models. In IJCAI, 2007.

[95] S. Lacoste-Julien, F. Husz ́ar, and Z. Ghahramani. Approximate inference for the loss-calibrated bayesian. In AISTATS.

[96] P. S. Laplace. Memoir on the probability of the causes of events. Statistical Science, 1(3), 1986.

[97] M. L ́azaro-Gredilla, S. Van Vaerenbergh, and N. D. Lawrence. Overlapping mixtures of gaussian processes for the data association problem. Pattern Recognition, 45(4), 2012.

[98] S. Levine. Reinforcement learning and control as probabilistic inference: Tutorial and review. arXiv:1805.00909, 2018.

[99] S. Levine and V. Koltun. Variational policy search via trajectory optimization. In NIPS, 2013.

[100] Y.Z. Li. Approximate Inference: New Visions. PhD thesis, 2018.

[101] Y.Z. Li, J.M. Hern ́andez-Lobato, and R.E. Turner. Stochastic expectation propagation. In NIPS, 2015.

[102] Y.Z. Li and Q. Liu. Wild variational approximations. In NIPS WS, 2016.

[103] Y.Z Li and r. e. Turner. R ́enyi divergence variational inference. In NIPS, 2016.

[104] Y.Z Li, M. Rowland, T. Bui, D. Hernandez-Lobato, and R. Turner. Black-Box Alpha Divergence Minimization. In ICML, 2016.

[105] Y.Z. Li, R. E. Turner, and Q. Liu. Approximate inference with amortised MCMC. arXiv:1702.08343, 2017.

[106] Wu. Lin, N. Hubacher, and M.E. Khan. Variational message passing with structured inference networks. arXiv:1803.05589, 2018.

[107] Q. Liu and Y. Feng. Two methods for wild variational inference. arXiv:1612.00081, 2016.

[108] Q. Liu, J. Lee, and M. Jordan. A kernelized stein discrepancy for goodness-of-fit tests. In ICML, 2016.

[109] Q. Liu and D. Wang. Stein variational gradient descent: A general purpose bayesian inference algorithm. In NIPS, 2016.

[110] Y. Liu, P. Ramachandran, Q. Liu, and J. Peng. Stein variational policy gradient. In UAI, 2017.

[111] C. J. Maddison, A. Mnih, and Y. W. Teh. The concrete distribution: A continuous relaxation of discrete random variables, 2017.

[112] S. Mandt and D. Blei. Smoothed gradients for stochastic variational inference. In NIPS, 2014.

[113] S. Mandt, M. D. Hoffman, and D. M. Blei. A Variational Analysis of Stochastic Gradient Algorithms. In ICML, 2016.

[114] S. Mandt, M. D. Hoffman, and D. M. Blei. Stochastic gradient descent as approximate bayesian inference. JMLR, 2017.

[115] S. Mandt, J. McInerney, F. Abrol, R. Ranganath, and Blei. Variational Tempering. In AISTATS, 2016.

[116] Joseph Marino, Yisong Yue, and Stephan Mandt. Iterative amortized inference. In ICML, 2018.

[117] J. McInerney, R. Ranganath, and D. Blei. The population posterior and bayesian modeling on streams. In NIPS, 2015.

[118] M.E.Khan, D. Nielsen, V. Tangkaratt, W. Lin, Y. Gal, and A. Srivastava. Fast and scalable bayesian deep learning by weight-perturbation in adam. arXiv:1806.04854, 2018.

[119] L. Mescheder, S. Nowozin, and A. Geiger. Adversarial variational bayes: Unifying variational autoencoders and generative adversarial networks. In ICML, 2017.

[120] L. Mescheder, S. Nowozin, and A. Geiger. The numerics of gans. In NIPS, 2017.

[121] M. M ́ezard, G. Parisi, and M. A. Virasoro. Spin glass theory and beyond. 1990.

[122] Y. Miao, L. Yu, and P. Blunsom. Neural variational inference for text processing. In ICML, 2016.

[123] A. C. Miller, N. J. Foti, A. D’Amour, and R. P. Adams. Reducing reparameterization gradient variance. In NIPS, 2017.

[124] A.C. Miller, N. Foti, and R. P. Adams. Variational boosting: Iteratively refining posterior approximations. In ICML, 2017.

[125] T. Minka. Power EP. Technical report, Technical report, Microsoft Research, Cambridge, 2004.

[126] T. Minka, J. M. Winn, J. P. Guiver, S. Webster, Y. Zaykov, B. Yangel, A. Spengler, and J. Bronskill. Infer.NET 2.6. http://research.microsoft.com/infernet, 2014.

[127] T. P. Minka. Expectation propagation for approximate bayesian inference. In UAI, 2001.

[128] T. P. Minka. Divergence measures and message passing. In Microsoft Research Technical Report, 2005.

[129] S. Mohamed and B. Lakshminarayanan. Learning in implicit generative models. arXiv:1610.03483, 2016.

[130] A. M  ̈uller. Integral probability metrics and their generating classes of functions. Advances in Applied Probability, 1997.

[131] K. P. Murphy, Y. Weiss, and M. I. Jordan. Loopy belief propagation for approximate inference: An empirical study. In UAI, 1999.

[132] C. Naesseth, F. Ruiz, S. Linderman, and D. M. Blei. Reparameterization gradients through acceptance-rejection sampling algorithms. In AISTATS, 2017.

[133] S. Nakajima and S. Watanabe. Variational bayes solution of linear neural networks and its generalization performance. Neural Computation, 19(4), 2007.

[134] E. Nalisnick and P. Smyth. Stick-breaking variational autoencoders. In ICLR, 2017.

[135] R. Nallapati, W. Cohen, and J. Lafferty. Parallelized variational em for latent dirichlet allocation: An experimental evaluation of speed and scalability. In ICDM WS, 2007.

[136] R. M. Neal. Probabilistic inference using markov chain monte carlo methods. In Technical Report, 1993.

[137] R. M. Neal. Bayesian learning for neural networks, volume 118. Springer Science & Business Media, 2012.

[138] W. Neiswanger, C. Wang, and E. Xing. Embarrassingly parallel variational inference in nonconjugate models. arXiv:1510.04163, 2015.

[139] S. Nowozin. Debiasing evidence approximations: On importance-weighted autoencoders and jackknife variational inference. In ICLR, 2018.

[140] C.J. Oates, M. Girolami, and N. Chopin. Control functionals for monte carlo integration. Journal of the Royal Statistical Society, 79(3):695–718, 2017.

[141] M. Opper, M. Fraccaro, U. Paquet, A. Susemihl, and O. Winther. Perturbation theory for variational inference. NIPS WS, 2015.

[142] M. Opper, U. Paquet, and O. Winther. Perturbative corrections for approximate inference in gaussian latent variable models. JMLR, 14(1), 2013.

[143] M. Opper and D. Saad. Advanced mean field methods: Theory and practice. MIT press, 2001.

[144] M. Opper and O. Winther. A mean field algorithm for bayes learning in large feed-forward neural networks. In NIPS, 1996. 2.

[145] M. Opper and O. Winther. Tractable approximations for probabilistic models: The adaptive thouless-andersonpalmer mean field approach. Physical Review Letters, 86(17), 2001.

[146] J. Paisley, D. M. Blei, and M. I. Jordan. Variational bayesian inference with stochastic search. In ICML, 2012.

[147] G. Parisi. Statistical field theory. Addison-Wesley, 1988.

[148] D Perekrestenko, V Cevher, and M Jaggi. Faster coordinate descent via adaptive importance sampling. In AISTATS, 2017.

[149] C. Peterson and J. R. Anderson. A mean field theory learning algorithm for neural networks. Complex systems, 1, 1987.

[150] T Plefka. Convergence condition of the TAP equation for the infinite-ranged ising spin glass model. Journal of Physics A: Mathematical and general, 15(6), 1982.

[151] I. Porteous, D. Newman, A. Ihler, A. Asuncion, P. Smyth, and M. Welling. Fast collapsed gibbs sampling for latent dirichlet allocation. 2008.

[152] T. Rainforth, R. Cornish, H. Yang, A. Warrington, and F. Wood. On Nesting Monte Carlo Estimators. In ICML, 2018.

[153] T. Rainforth, A. R. Kosiorek, T. A. Le, C. J. Maddison, M. Igl, F. Wood, and Y. W. Teh. Tighter variational bounds are not necessarily better. In ICML, 2018.

[154] R. Ranganath, S. Gerrish, and D. Blei. Black box variational inference. In AISTATS, 2014.

[155] R. Ranganath, D. Tran, J. Altosaar, and D. M. Blei. Operator variational inference. In NIPS. 2016.

[156] R. Ranganath, D. Tran, and D. M. Blei. Hierarchical variational models. In ICML, 2016.

[157] R. Ranganath, C. Wang, D. Blei, and E. Xing. An adaptive learning rate for Stochastic Variational Inference. In ICML, 2013.

[158] J. Raymond, A. Manoel, and M. Opper. Expectation propagation. arXiv:1409.6179, 2014.

[159] D. Rezende and S. Mohamed. Variational Inference with Normalizing Flows. In ICML, 2015.

[160] D. J. Rezende, S. Mohamed, and D. Wierstra. Stochastic backpropagation and approximate inference in deep generative models. In ICML, 2014.

[161] H. Robbins and S. Monro. A stochastic approximation method. The annals of mathematical statistics, 1951.

[162] G. Roeder, Y. Wu, and D. Duvenaud. Sticking the landing: An asymptotically zero-variance gradient estimator for variational inference. In NIPS, 2017.

[163] J. T. Rolfe. Discrete variational autoencoders. In ICLR, 2017.

[164] K. Rose, E. Gurewitz, and G. Fox. A deterministic annealing approach to clustering. Pattern Recognition Letters, 11(9), 1990.

[165] S. M. Ross. Simulation. Elsevier, 2006.

[166] F. J.R. Ruiz, M. K. Titsias, and D. M. Blei. The generalized reparameterization gradient. In NIPS. 2016.

[167] F. J.R. Ruiz, M. K. Titsias, and D. M. Blei. Overdispersed black-box variational inference. In UAI, 2016.

[168] A. Saeedi, M. D. Hoffman, S. J. DiVerdi, A. Ghandeharioun, M. J. Johnson, and R. P. Adams. Multimodal prediction and personalization of photo edits with deep generative models. arXiv:1704.04997, 2017.

[169] T. Salimans, D. P. Kingma, and M. Welling. Markov chain monte carlo and variational inference: Bridging the gap. In ICML, 2015.

[170] T. Salimans and D. A. Knowles. Fixed-form variational posterior approximation through stochastic linear regression. Bayesian Analysis, 8(4), 2013.

[171] M. Sato. Online model selection based on the variational Bayes. Neural Computation, 13(7):1649–1681, 2001.

[172] L. K. Saul, T. Jaakkola, and M. I. Jordan. Mean field theory for sigmoid belief networks. Journal of artificial intelligence research, 4(1), 1996.

[173] J. Schulman, S. Levine, P. Abbeel, M. Jordan, and P. Moritz. Trust region policy optimization. In ICML, 2015.

[174] M. Seeger. Expectation propagation for exponential families. Technical report, 2005.

[175] A. Shah, D. A. Knowles, and Z. Ghahramani. An Empirical Study of Stochastic Variational Algorithms for the Beta Bernoulli Process. In ICML, 2015.

[176] J. Shi, J. Chen, J. Zhu, S. Sun, Y. Luo, Yihong Gu, and Yuhao Zhou. Zhusuan: A library for bayesian deep learning. arXiv:1709.05870, 2017.

[177] E. Snelson and Z. Ghahramani. Sparse gaussian processes using pseudo-inputs. NIPS, 18, 2006.

[178] C. K. Sønderby, T. Raiko, L. Maaløe, S. K. Sønderby, and O. Winther. How to train deep variational autoencoders and probabilistic ladder networks. In ICML, 2016.

[179] B. K. Sriperumbudur, K. Fukumizu, A. Gretton, B. Sch  ̈olkopf, and G. RG. Lanckriet. On integral probability metrics,\phi-divergences and binary classification. arXiv preprint arXiv:0901.2698, 2009.

[180] A. Srivastava and C. Sutton. Autoencoding variational inference for topic models. In ICLR, 2017.

[181] C. Stein. A bound for the error in the normal approximation to the distribution of a sum of dependent random variables. In Berkeley Symposium on Mathematical Statistics and Probability, 1972.

[182] J. Sung, Z. Ghahramani, and S. Y. Bang. Latent-space variational bayes. TPAMI, 30(12), 2008.

[183] R. S. Sutton and A. G. Barto. Reinforcement learning: An introduction, volume 1. MIT press Cambridge, 1998.

[184] L. SL. Tan. Stochastic variational inference for largescale discrete choice models using adaptive batch sizes. Statistics and Computing, 27(1):237–257, 2017.

[185] T. Tanaka. Estimation of third-order correlations within mean field approximation. In NIPS, 1998.

[186] T. Tanaka. A theory of mean field approximation. In NIPS, 1999.

[187] T. Tanaka. Information geometry of mean-field approximation. Neural Computation, 12(8), 2000.

[188] Y. W. Teh, D. Newman, and M. Welling. A collapsed variational Bayesian inference algorithm for latent Dirichlet allocation. In NIPS, 2006.

[189] L. Theis and M. Hoffman. A Trust-region Method for Stochastic Variational Inference with Applications to Streaming Data. In ICML, 2015.

[190] D.J. Thouless, P. W. Anderson, and R. G. Palmer. Solution of ’solvable model of a spin glass’. Philosophical Magazine, 35(3), 1977.

[191] T. Tieleman and G. Hinton. Lecture 6.5-rmsprop: Divide the gradient by a running average of its recent magnitude. COURSERA: Neural networks for machine learning, 4(2), 2012.

[192] L. Tierney and J.B. Kadane. Accurate approximations for posterior moments and marginal densities. Journal of the american statistical association, 81(393), 1986.

[193] N. Tishby, F. C. Pereira, and W. Bialek. The information bottleneck method. physics/0004057, 2000.

[194] M. Titsias and M. L ́azaro-Gredilla. Local Expectation Gradients for Black Box Variational Inference. In NIPS, 2015.

[195] M. K. Titsias. Variational learning of inducing variables in sparse gaussian processes. In AISTATS, 2009.

[196] M. K. Titsias. Learning model reparametrizations: Implicit variational inference by fitting mcmc distributions. arXiv:1708.01529, 2017.

[197] M.K. Titsias and M. L ́azaro-Gredilla. Variational heteroscedastic gaussian process regression. In ICML, 2011.

[198] D. Tolpin, J. W. van de Meent, H. Yang, and F. Wood. Design and implementation of probabilistic programming language anglican. arXiv:1608.05263, 2016. 2.

[199] D. Tran, D. M. Blei, and E. M. Airoldi. Copula variational inference. In NIPS, 2015.

[200] D. Tran, A. Kucukelbir, A. B. Dieng, M. Rudolph, D. Liang, and D. M. Blei. Edward: A library for probabilistic modeling, inference, and criticism. arXiv:1610.09787, 2016.

[201] D. Tran, R. Ranganath, and D. M. Blei. The Variational Gaussian Process. stat, 1050, 2016.

[202] D. Tran, R. Ranganath, and D. M. Blei. Hierarchical implicit models and likelihood-free variational inference. arXiv:1702.08896, 2017.

[203] G. Tucker, A. Mnih, C.J. Maddison, J. Lawson, and J. Sohl-Dickstein. REBAR: Low-variance, unbiased gradient estimates for discrete latent variable models. In NIPS, 2017.

[204] M. J. Wainwright, T. S. Jaakkola, and A. S. Willsky. A new class of upper bounds on the log partition function. IEEE Transactions on Information Theory, 51(7), 2005.

[205] M. J. Wainwright and M. I. Jordan. Graphical models, exponential families, and variational inference. Foundations and Trends R©in Machine Learning, 1(1-2), 2008.

[206] C. Wang, D. Blei, and L. Fei-Fei. Simultaneous image classification and annotation. In CVPR, 2009.

[207] C. Wang and D. M. Blei. Variational inference in nonconjugate models. JMLR, 14(Apr), 2013.

[208] C. Wang, X. Chen, A. J. Smola, and E. P. Xing. Variance reduction for stochastic gradient optimization. In NIPS, 2013.

[209] C. Wang, J. Paisley, and D. M. Blei. Online variational inference for the hierarchical dirichlet process. In AISTATS, 2011.

[210] D. Wang and Q. Liu. Learning to draw samples: With application to amortized MLE for generative adversarial learning. arXiv:1611.01722, 2016.

[211] Y. Wang, G. Williams, E. Theodorou, and L. Song. Variational policy for guiding point processes. In ICML, 2017.

[212] Y. X. Wang, A. Kucukelbir, and D. M. Blei. Robust Probabilistic Modeling with Bayesian Data Reweighting. In ICML, 2017.

[213] Yixin Wang and David M Blei. Frequentist consistency of variational bayes. 2018.

[214] M. Welling and Y. W. Teh. Bayesian learning via stochastic gradient langevin dynamics. In ICML, 2011.

[215] R. J. Williams. Simple statistical gradient-following algorithms for connectionist reinforcement learning. Machine learning, 8(3-4):229–256, 1992.

[216] J. Winn and C. M. Bishop. Variational message passing. JMLR, 6, 2005.

[217] M. Yin and M. Zhou. Semi-implicit variational inference. In ICML, 2018.

[218] M.D. Zeiler. Adadelta: an adaptive learning rate method. arXiv:1212.5701, 2012.

[219] K. Zhai, J. Boyd-Graber, N. Asadi, and M. L. Alkhouja. Mr. lda: A flexible large scale topic modeling package using variational inference in mapreduce. In WWW, 2012.

[220] C. Zhang. Structured Representation Using Latent Variable Models. PhD thesis, KTH Royal Institute of Technology, 2016.

[221] C. Zhang, R. Bamler, M. Opper, and S. Mandt. Perturbative black box variational inference. In NIPS, 2017.

[222] C. Zhang, C. H. Ek, X. Gratal, F. T. Pokorny, and H. Kjellstr  ̈om. Supervised hierarchical Dirichlet processes with variational inference. In ICCV WS, 2013.

[223] C. Zhang, H. Kjellstrom, and S. Mandt. Determinantal point processes for mini-batch diversification. In UAI, 2017.

[224] C. Zhang, C.  ̈Oztireli, S. Mandt, and G. Salvi. Active mini-batch sampling using repulsive point processes. arXiv:1804.02772, 2018.

[225] P.L. Zhao and T. Zhang. Accelerating minibatch stochastic gradient descent using stratified sampling. arXiv:1405.3080, 2014.

[226] P.L. Zhao and T. Zhang. Stochastic optimization with importance sampling for regularized loss minimization. In ICML, 2015.

[227] S. Zhao, J. Song, and S. Ermon. Towards deeper understanding of variational autoencoding models. CoRR, abs/1702.08658, 2017.

[228] S.D. Zhe, K.C. Lee, K. Zhang, and J. Neville. Online spike-and-slab inference with stochastic expectation propagation. In NIPS WS, 2016.

[229] H. Zhu and R. Rohwer. Information geometric measurements of generalisation. Technical report, 1995.